Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cp -r /content/drive/MyDrive/TFM/IA/scripts/* /content/

Mounted at /content/drive


In [ ]:
!pip install -q fiftyone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.

In [3]:
import json
import time
import datetime

# Torch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchHelpers.engine import train_one_epoch, evaluate

import torchvision.transforms as T

from collections import Counter

# import fiftyone as fo

import numpy as np
from PIL import  ImageDraw
from PIL import  Image as PILImage
import matplotlib.pyplot as plt
import cv2

from enum import Enum

# Garbage collector and os operations
import gc
import os

# Constants

In [5]:
# Constants file

# Categories
CATEGORIES = ['staff']

CATEGORIES_TO_NUM = dict({
  'background': 0,
  'staff': 1,
  'empty-staff': 1,
})
NUM_TO_CATEGORIES = dict({
  0: 'background',
  1: 'staff',
})


# Datasets
DATASETS = [
  'Capitan',
  'SEILS',
  'FMT_C',
]


DEBUG_FLAG = False

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

BBOX_REDIMENSION = 0.8
BBOX_REDIMENSIONED_RECOVER = 1 / BBOX_REDIMENSION

TRAIN_NUM_EPOCHS = 100
TRAIN_PATIENTE = 20

DRIVE_IA_FOLDER = ''
DRIVE_IA_FOLDER = '/content/drive/MyDrive/TFM/IA'

DRIVE_DATASETS_FOLDER = f'{DRIVE_IA_FOLDER}/datasets'
DRIVE_MODELS_FOLDER = f'{DRIVE_IA_FOLDER}/models'
DRIVE_LOGS_FOLDER = f'{DRIVE_IA_FOLDER}/logs'
DRIVE_IMG_FOLDER = f'{DRIVE_IA_FOLDER}/img'

DRIVE_TRAIN_LOGS_FOLDER = f'{DRIVE_LOGS_FOLDER}/train'
DRIVE_VAL_LOGS_FOLDER = f'{DRIVE_LOGS_FOLDER}/val'
DRIVE_TEST_LOGS_FOLDER = f'{DRIVE_LOGS_FOLDER}/test'

DRIVE_VAL_IMG_FOLDER = f'{DRIVE_IMG_FOLDER}/val'
DRIVE_TEST_IMG_FOLDER = f'{DRIVE_IMG_FOLDER}/test'

SAE_IMAGE_SIZE =  (512, 512)

BIN_UMBRALS = [i/100 for i in range(10, 91, 5)]
DROPOUT_VALUES = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

class PredictionsCombinationType(Enum):
    NONE = ''
    MEAN = 'mean'
    MAX  = 'max'


# Models

In [6]:
class SAE(torch.nn.Module):
  def __init__(self):
    with_batch_normalization = True
    dropout_probability = 0

    super().__init__()
    self.encoder = torch.nn.Sequential(
      nn.Conv2d(in_channels=1  , out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.MaxPool2d(kernel_size=(2, 2)),

      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.MaxPool2d(kernel_size=(2, 2)),

      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.MaxPool2d(kernel_size=(2, 2)),
    )

    self.decoder = torch.nn.Sequential(
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.Upsample(scale_factor=(2, 2)),

      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.Upsample(scale_factor=(2, 2)),

      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 5), padding=2),
      nn.BatchNorm2d(128) if with_batch_normalization else nn.Identity(),
      nn.ReLU(),
      nn.Dropout2d(p = dropout_probability),
      nn.Upsample(scale_factor=(2, 2)),

      nn.Conv2d(in_channels=128, out_channels=1  , kernel_size=(5, 5), padding=2),
      nn.Sigmoid(),
    )

  def set_dropout_probability(self, dropout_probability = 0.2):
    for module in self.modules():
      if 'Dropout' in type(module).__name__:
        module.p = dropout_probability

  def enable_eval_dropout(self):
    for module in self.modules():
      if 'Dropout' in type(module).__name__:
        print(module)
        module.train()

  def forward(self, x):
    # Checks if input if batched or not. If it's not batched, add a dimension
    addDimension = x.dim() == 3

    if addDimension:
      x = torch.stack([x])

    x = self.encoder(x)
    x = self.decoder(x)

    # If we added a dimension, remove it for the loss function
    if addDimension:
      x = x.squeeze(0)

    return x

In [7]:
class ModelCheckpoint:
  def __init__(self, filepath, model):
    """
      Initialises the class for a ModelCheckpoint callback.
      Arguments:
        filepath: string
          filename with .pt extension to save the model
        model: torch.nn
          model to save while training
    """
    self.filepath = filepath
    self.model: torch.nn.Module = model
    self.save()

  def save(self):
    torch.save(self.model, self.filepath)


class EarlyStopping:
  def __init__(self, patience, epochs, checkpoint, epsilon, searchFor = 'min', plotMsg = False):
    """
      Initialises the class for an Early Stopping callback.
      Arguments:
        patience: int
          number of epochs without improving the metric
        epochs: int
          total number of epochs
        checkpoint: ModelCheckpoint
          object for saving the best model
        epsilon: float
          threshold in order to update best result
        searchFor: str
          if best result is min or max
    """
    self.patience = patience
    self.epochs = epochs
    self.checkpoint = checkpoint
    self.stop_training = False
    self.actual_patience = 1
    self.best_result = None
    self.epsilon = epsilon
    self.plotMsg = plotMsg
    self.searchFor = searchFor

    if self.searchFor == 'min':
      self.best_result = float('inf')
    elif self.searchFor == 'max':
      self.best_result = 0.0

  def update(self, last_result, epoch):
    """
      Updates results after each epoch and saves the best model
      through ModelCheckpoint Object.
      Arguments:
        last_result: float
          result obtained on the selected metric
      Returns:
        Boolean
          True if the current patience has achieved maximum patience
          False otherwise
    """
    hasImproved = False

    if self.searchFor == 'min':
      hasImproved =  last_result < self.best_result
    elif self.searchFor == 'max':
      hasImproved =  last_result > self.best_result

    if hasImproved:
      # Improvement
      self.actual_patience = 1
      last_best = self.best_result
      self.best_result = last_result

      # Saving best model
      self.checkpoint.save()
      if self.plotMsg:
        print(f'Updating best result ({self.best_result}), last: {last_best}. Difference of {np.round(abs(last_best-self.best_result), 3)}.')
        print(f'Saving on {self.checkpoint.filepath}')

    else:
      # No improvement
      self.actual_patience += 1
      if self.actual_patience >= self.patience:
        self.stop_training = True
        if self.plotMsg:
          print(f'Training stopped with patience {self.patience}')
          print(f'Best result obtained: {np.round(self.best_result, 3)}')
      if self.plotMsg:
        print(f'Not updating best result ({self.best_result}), last: {last_result}.')


    # if self.plotMsg:
    print(f'\tEarly stopping: [{self.actual_patience}/{self.patience}] on epoch {epoch}/{self.epochs}')

    return self.stop_training



# Classes

In [9]:
class Symbol():
    def __init__(self, boxes : list[int], position_in_staff :str, agnostic_symbol_type:str) -> None:
        self.box = boxes
        self.position_in_staff = position_in_staff
        self.agnostic_symbol_type = agnostic_symbol_type

    def __str__(self) -> str:
        return f'Boxes: {self.box}\t, Position: {self.position_in_staff}\t, Symbol: {self.agnostic_symbol_type}'

    def resize(self, ratio_width: float, ratio_height: float):
        box = self.box
        self.box = [int(box[0]/ratio_width), int(box[1]/ratio_height), int(box[2]/ratio_width), int(box[3]/ratio_height)]

        return self


class Region():
    def __init__(self, box: list[int], label: int, notes: list[Symbol] = None) -> None:
        self.box = box
        self.label = label
        self.notes = notes

    def __str__(self, symbols = True) -> str:
        message = f'Bounding boxes: {self.box}\t, Class: {NUM_TO_CATEGORIES[self.label]}'

        if symbols:
            message += '\t, Symbols: '
            if self.notes is None:
                message += '[]'
            else:
                message += '\n\t\t['
                for sym in self.notes:
                    message += f'\n\t\t\t{sym}'
                message += '\n\t\t]'

        return message

    def isEmpty(self):
        return len(self.box) == 0

    def resize(self, ratio_width: float, ratio_height: float):

        box = self.box
        self.box = [int(box[0]/ratio_width), int(box[1]/ratio_height), int(box[2]/ratio_width), int(box[3]/ratio_height)]

        if self.notes is not None:
            self.notes = [note.resize(ratio_width=ratio_width, ratio_height=ratio_height) for note in self.notes]

        return self

class ImageExample():
    def __init__(self, regions: list[Region], image: PILImage, imageName: str, imagePath: str) -> None:
        self.regions = regions
        self.image: PILImage = image
        self.imageName = imageName
        self.imagePath = imagePath

    def isEmpty(self):
        return len(self.regions) <= 0

    def dataAsTensor(self):
        if self.isEmpty(): # If image does not contain any object (background)
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros(1, dtype=torch.int64)
        else:
            boxes = torch.as_tensor([region.box for region in self.regions], dtype=torch.float32) # [xmin, ymin, xmax, ymax]
            labels = torch.as_tensor([region.label for region in self.regions], dtype=torch.int64)

        return boxes, labels

    def get_boxes(self):
        return [region.box for region in self.regions]

    def get_name(self):
        return self.imageName

    def resize_examples(self, resize_shape):

        if resize_shape is not None:
            width, height = self.image.size
            ratio_width = width / resize_shape[0]
            ratio_height = height / resize_shape[1]

            self.image = self.image.resize(resize_shape)

            self.regions = [region for region in self.regions if not region.isEmpty()]
            self.regions = [region.resize(ratio_width=ratio_width, ratio_height=ratio_height) for region in self.regions]
            # for region in self.regions: region.resize(ratio_width=ratio_width, ratio_height=ratio_height)

    def __str__(self) -> str:
        for r in self.regions:
            print(r)

# Dataset Utils

## Read JSON functions

In [10]:
def read_json_symbols(box: json, resize_shape, ratio_width, ratio_height) -> list[Symbol]:
    """Reads the Symbols from the JSON and returns them all as a list

    Args:
        box (json): json following the symbols section
        resize_shape (list): None if no reshape is needed
        ratio_width (float): width ratio to resize
        ratio_height (float): height ratio to resize

    Returns:
        list[Symbol]: list of Symbols read
    """
    listOfSymbols = None

    if 'symbols' in box:
        listOfSymbols = []
        symbols = box['symbols']

        for symbol in symbols:
            symbolBox = symbol['bounding_box']
            symbolType = symbol['agnostic_symbol_type']
            symbolPos = symbol['position_in_staff']

            xmin, xmax = symbolBox['fromX'], symbolBox['toX']
            ymin, ymax = symbolBox['fromY'], symbolBox['toY']

            if resize_shape is not None:
                xmin, xmax = int(round(xmin/ratio_width, 0)), int(round(xmax/ratio_width, 0))
                ymin, ymax = int(round(ymin/ratio_height, 0)), int(round(ymax/ratio_height, 0))

            listOfSymbols.append(Symbol([xmin, ymin, xmax, ymax], symbolPos, symbolType))

    return listOfSymbols

def read_json_datafile(path_json: str, path_image: str, resize_shape = None, read_symbols = False) :
    """Reads the JSON data file and returns a list of Regions

    Args:
        path_json (str): path to JSON data file to read
        path_image (str): path to image to load
        resize_shape (list(int), optional): shape of reshape. Defaults to None.
        read_symbols (bool, optional): if Symbols must be read. Defaults to False.

    Returns:
        PILImage, list(Region): image and regions from that image
    """

    # print(f'Path_image: {path_image}')
    # print(f'Path_json: {filename}')

    image = PILImage.open(path_image)#.convert("RGB")
    width, height = image.size

    ratio_width, ratio_height = 0, 0
    if resize_shape is not None:
        image = image.resize(resize_shape)
        ratio_width = width / resize_shape[0]
        ratio_height = height / resize_shape[1]

    # getBoxesFunction = lambda b, rW, rH : [int(b['fromX']), int(b['fromY']), int(b['toX']), int(b['toY'])] if resize_shape is None else [int(b['fromX']/rW), int(b['fromY']/rH), int(b['toX']/rW), int(b['toY']/rH)]
    # getCategoriInt = lambda c : 'staff' if 'staff' in c else c
    # readSymbols = lambda b, r, rW, rH : read_json_symbols(box=b, resize_shape=r, ratio_width=rW, ratio_height=rH)
    # createRegionSymbols = lambda b, r, rH, rW : Region(box=getBoxesFunction(b=b['bounding_box'], rH=rH, rW=rW), label=getCategoriInt(b['type']), notes=readSymbols(b=b, r=r, rH=rH, rW=rW))
    # createRegionNonSymbols = lambda b, rH, rW : Region(box=getBoxesFunction(b=b['bounding_box'], rH=rH, rW=rW), label=getCategoriInt(b['type']), notes=None)

    boxes = []
    classes = []
    regionsList: list[Region] = []
    with open(path_json) as f:
        # Read image from json (json info and image)
        example_dict = json.load(f)
        #filename_image = example_dict['filename'].encode('utf8')
        if "pages" not in example_dict:
            return ImageExample(regions=[Region(boxes, classes)], image=image, imageName=get_file_name(path_image), imagePath=path_image)

        pages = example_dict['pages']

        for _, page in enumerate(pages):
            if "regions" not in page:
                continue
            regions = page['regions']

            # if read_symbols:
            #     regionsList.append([createRegionSymbols(b=box, r=resize_shape, rH=ratio_height, rW=ratio_width) for box in regions])
            # else:
            #     regionsList.append([createRegionNonSymbols(b=box, rH=ratio_height, rW=ratio_width) for box in regions])

            for box in regions:
                box_data = box['bounding_box']
                category = box['type']
                if "staff" in category: # empty-staff => staff
                    category = "staff"

                # skip that aren't interesting categories (author, etc)
                if category in CATEGORIES:
                    # print(f'region: {box}')
                    category_int = CATEGORIES_TO_NUM[category]
                    xmin, xmax = box_data['fromX'], box_data['toX']
                    ymin, ymax = box_data['fromY'], box_data['toY']

                    if resize_shape is not None:
                        xmin, xmax = xmin/ratio_width, xmax/ratio_width
                        ymin, ymax = ymin/ratio_height, ymax/ratio_height

                    boxes_images = [int(xmin), int(ymin), int(xmax), int(ymax)]

                    if xmax <= xmin or ymax <= ymin:
                        continue

                    boxes_images = [int(box) for box in boxes_images]

                    if len(boxes_images) == 4:
                        if category == "staff" and read_symbols:
                            listOfSymbols = read_json_symbols(box=box, resize_shape=resize_shape, ratio_width=ratio_width, ratio_height=ratio_height)
                            regionsList.append(Region(box = boxes_images, label = category_int, notes = listOfSymbols))
                        else:
                            regionsList.append(Region(box = boxes_images, label = category_int))

    return ImageExample(regions=regionsList, image=image, imageName=get_file_name(path_image), imagePath=path_image)

def read_paths_dataset_files(path_listJsons: str):
    """Read a file that contains a list to the paths to different JSON files

    Args:
        path_listJsons (str): path to file containing the list of json files

    Returns:
        [jsonList], [imagesList]: paths to different images and JSON data file
    """
    imagesList: list[str] = []
    jsonList: list[str] = []

    fileReading = open(path_listJsons, 'r')
    path = path_listJsons[:path_listJsons.rfind('/')+1]

    for file in fileReading:
        singleFileName = file[:file.rfind('.')]
        imagesList.append(path + 'images/' + singleFileName)
        jsonList.append(path + 'JSON/' + file[:file.rfind('n')+1])  # Removes empty spaces from end

    return jsonList, imagesList

def pathExist(dir_path):
    return os.path.exists(dir_path)

def makeDirIfNotExist(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

## Dataset loader

Reads the dataset files and load the images/bounding boxes from it

In [11]:
def load_dataset(imagesPath: list[str], jsonsPath: list[str], reshape = None):
    """_summary_

    Args:
        imagesPath (list[str]): paths to images
        jsonsPath (list[str]): paths to JSON data files
        reshape (list[int], optional): images size reshape if needed. Defaults to None.

    Returns:
        tuple[list[PILImage], list[list[Region]]]: lists of images and regions for each image
    """

    return [read_json_datafile(iPath, jPath, reshape) for iPath, jPath in zip(imagesPath, jsonsPath)]

def get_file_name(filePath: str):
    return filePath[filePath.rfind('/')+1:filePath.find('.')]

def draw_back_white_bb_image(height, width, boxes):
    targetImage = PILImage.new("RGB", (width, height), "#000000")

    draw = ImageDraw.Draw(targetImage)
    for box in boxes:
        draw.rectangle([box[0], box[1], box[2]-1, box[3]-1], fill="#FFFFFF")

    return targetImage

def draw_boxes_on_image(example: ImageExample, saveDirectory: str):
    """Save image with passes boxes drawn

    Args:
        image (Image): image to save
        boxes (list[list[int]]): bounding boxes to save
        imageName (str): image name
        saveDirectory (str): directory to save
    """
    # Read file & normalize image
    #img, regionsList = read_json_datafile(path_json, path_img, reshape)
    img = example.image

    print(saveDirectory)

    makeDirIfNotExist(saveDirectory)

    drawingImage = ImageDraw.Draw(img)

    # Save image
    # for box in example.get_boxes():
    #     drawingImage.rectangle(box, outline="red", width=3)
    [drawingImage.rectangle(box, outline="red") for box in example.get_boxes()]

    img.save(saveDirectory + example.imageName + '.jpg')

def get_transform():
    t = [
        T.ToTensor(),
        T.Grayscale(),
        ]

    return T.Compose(t)


def resize_box(box, vResize, hResize):
    """
      Boxes as [x1, y1, x2, y2]
    """
    if(len(box)) < 4:
      print(box)
    # Get dimensions
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    w = x2 - x1 # 100
    h = y2 - y1 # 100

    # Calculate resized dimensions
    w2 = w * hResize # 80
    h2 = h * vResize # 80

    # Get size difference
    wDif = w2 - w  # 100 - 80 = 20
    hDif = h2 - h  # 100 - 80 = 20

    # Get how much each coordinate must move
    xMov = wDif / 2
    yMov = hDif / 2

    # Get new coordinates
    x1 = x1 - xMov
    x2 = x2 + xMov

    y1 = y1 - yMov
    y2 = y2 + yMov

    return [x1, y1, x2, y2]

class DatasetLoader:
    def __init__(self, dataset_name: str, reshape:list[float] = None) -> None:
        """Create class that loads the dataset

        Args:
            dataset_name (str): forder path to dataset containing train, test and val's txt files without final / (Ex.: "datasets/Captitan")
            reshape: reshape of the images
        """
        self.datasetFolder = f'{DRIVE_DATASETS_FOLDER}/{dataset_name}'
        self.reshape = reshape

    def loadTrainPaths(self):
        return read_paths_dataset_files(f'{self.datasetFolder}/train.txt')

    def loadTestPaths(self):
        return read_paths_dataset_files(f'{self.datasetFolder}/test.txt')

    def loadValPaths(self):
        return read_paths_dataset_files(f'{self.datasetFolder}/val.txt')

    def loadTrainDataset(self):
        trainJSonPaths, trainImagesPath = self.loadTrainPaths()
        return [read_json_datafile(json, img, self.reshape) for json, img in zip(trainJSonPaths, trainImagesPath)]

    def loadValDataset(self):
        valJSonPaths, valImagesPath = self.loadValPaths()
        return [read_json_datafile(json, img, self.reshape) for json, img in zip(valJSonPaths, valImagesPath)]

    def loadTestDataset(self):
        testJSonPaths, testImagesPath = self.loadTestPaths()
        return [read_json_datafile(json, img, self.reshape) for json, img in zip(testJSonPaths, testImagesPath)]

    def loadAllDatasetPaths(self):
        return self.loadTrainPaths(), self.loadValPaths(), self.loadTestPaths()

    def loadAllDataset(self):
        return self.loadTrainDataset(), self.loadValDataset(), self.loadTestDataset()

    def drawBoxesInDataset(self):
        train, val, test = self.loadAllDataset()
        [draw_boxes_on_image(example=example, saveDirectory=self.datasetFolder+"/train/") for example in train]
        [draw_boxes_on_image(example=example, saveDirectory=self.datasetFolder+"/val/") for example in val]
        [draw_boxes_on_image(example=example, saveDirectory=self.datasetFolder+"/test/") for example in test]

## Dataset holders

In [12]:
class TrainMusicDataset(torch.utils.data.Dataset):
    def __init__(self, name, jsonPaths: list[str], imagesPaths: list[str], resize_shape = None, transforms = get_transform(), box_resize_vertical = False, box_resize_horizontal = False):
        """
        Arguments:
            name: Name of the dataset (b-59-80, Seils, Mus-Trad-...)
            path_json: list with all paths to json
            path_json: list with all paths to source images
        """
        self.name = name
        self.resize_shape = resize_shape
        self.jsonPaths: list[str] = jsonPaths
        self.imagesPaths: list[str] = imagesPaths
        self.transforms = transforms
        self.box_resize_vertical = box_resize_vertical
        self.box_resize_horizontal = box_resize_horizontal


    def __getitem__(self, idx):
        path_img = self.imagesPaths[idx]
        path_json = self.jsonPaths[idx]

        example: ImageExample = read_json_datafile(path_json, path_img, self.resize_shape)

        # example: ImageExample = self.imagesPaths[idx]

        example.resize_examples(self.resize_shape)

        boxes, labels = example.dataAsTensor()
        img = example.image

        # image_id = torch.tensor([idx])
        # area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        draw_boxes = example.get_boxes()
        if self.box_resize_vertical or self.box_resize_horizontal:
            vResize = BBOX_REDIMENSION if self.box_resize_vertical else 1
            hResize = BBOX_REDIMENSION if self.box_resize_horizontal else 1
            draw_boxes = [resize_box(box, vResize = vResize, hResize = hResize) for box in draw_boxes]

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["name"]  = example.get_name()

        width, height = img.size
        targetImage = draw_back_white_bb_image(height=height, width=width, boxes=draw_boxes)

        if self.transforms is not None:
            img = self.transforms(img)
            targetImage = self.transforms(targetImage)

        return img, target, targetImage# torch.tensor([img]), [target]

    def getImagesPaths(self):
        return self.imagesPaths

    def __len__(self):
        return len(self.imagesPaths)

class TestMusicDataset(torch.utils.data.Dataset):
    def __init__(self, name, jsonPaths: list[str], imagesPaths: list[str], resize_shape = None, transforms = get_transform()):
        """
        Arguments:
            name: Name of the dataset (b-59-80, Seils, Mus-Trad-...)
            path_json: list with all paths to json
            path_json: list with all paths to source images
        """
        self.name = name
        self.resize_shape = resize_shape
        self.jsonPaths: list[str] = jsonPaths
        self.imagesPaths: list[str] = imagesPaths
        self.transforms = transforms


    def __getitem__(self, idx):
        path_img = self.imagesPaths[idx]
        path_json = self.jsonPaths[idx]

        example: ImageExample = read_json_datafile(path_json, path_img, self.resize_shape)

        example.resize_examples(self.resize_shape)

        boxes, _ = example.dataAsTensor()
        img = example.image

        if self.transforms is not None:
            #img, target = self.transforms(img, target)
            img = self.transforms(img)

        return img, boxes# torch.tensor([img]), [target]

    def getImagesPaths(self):
        return self.imagesPaths

    def __len__(self):
        return len(self.imagesPaths)

# IoU and CC Functions

In [13]:
import numpy as np

"""IoU and F1"""
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Args:
        box1 (list): Bounding box 1 in the format [x1, y1, x2, y2].
        box2 (list): Bounding box 2 in the format [x1, y1, x2, y2].

    Returns:
        float: IoU value.
    """
    # Get points of boxes
    x11, y11, x12, y12 = box1
    x21, y21, x22, y22 = box2

    # Calculate dimensions of boxes
    w1, h1 = x12 - x11, y12 - y11
    w2, h2 = x22 - x21, y22 - y21

    # Calculate areas of both boxes
    area_box1: float = w1 * h1
    area_box2: float = w2 * h2

    # Calculate coordinates of the intersection rectangle
    x_intersection = max(x11, x21)
    y_intersection = max(y11, y21)
    w_intersection = max(0, min(x12, x22) - x_intersection)
    h_intersection = max(0, min(y12, y22) - y_intersection)

    # Calculate area of intersection
    area_intersection: float = w_intersection * h_intersection

    # Calculate area of union
    area_union: float = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    return (area_intersection / area_union) if (area_union > 0.0) else 0.0

def calculate_F1(y_true, y_pred, iou_threshold: float =0.5):
    """Calculate F1 score

    Args:
        y_true (list): List of true bounding boxes
        y_pred (list): List of predicted bounding boxes
        iou_threshold (float): IoU Threshold to math prediction with true. Defaults to 0.5.

    Returns:
        float: F1 score
        list: list of matched IoUs
    """
    num_preds = len(y_pred)
    num_true  = len(y_true)

    if num_preds == 0 and num_true == 0:
        return 1, []

    true_positives  = 0.0
    false_positives = num_preds
    false_negatives = num_true

    matrix_of_iou = np.asarray([[calculate_iou(box_pred, box_true) for box_true in y_true] for box_pred in y_pred])
    matched_ious  = []

    if DEBUG_FLAG:
      print(f'Matrix of IoU:\n{matrix_of_iou}')

    while matrix_of_iou.size > 0:
        # Find the maximum IoU in the matrix
        current_max_IoU = np.max(matrix_of_iou)
        if current_max_IoU < iou_threshold:
            break

        # Add the current max IoU to matched IoUs
        matched_ious.append(current_max_IoU)

        # This is a match, update TP, FP, FN
        true_positives  += 1
        false_negatives -= 1
        false_positives -= 1

        # Get the indices of the maximum IoU
        max_index = np.unravel_index(matrix_of_iou.argmax(), matrix_of_iou.shape)

        # Remove the matched bounding boxes (row and column) from the IoU matrix
        matrix_of_iou = np.delete(matrix_of_iou, max_index[1], axis=1)  # Remove column
        matrix_of_iou = np.delete(matrix_of_iou, max_index[0], axis=0)  # Remove row

    # Number of predictions that haven't been matched (false predictions)

    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if num_preds > 0 else 0.
    recall    = true_positives / (true_positives + false_negatives) if num_true  > 0 else 0.

    if DEBUG_FLAG:
      print(f'Precision: {precision} \t Recall: {recall}')

    # print(f'{precision} -- {recall}')
    f1score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1score, matched_ious

# Draw Functions

In [14]:
""" Draw functions """

def drawTensorImageColor(tensor_image):

  img = np.uint8(tensor_image.squeeze().permute(1, 2, 0).numpy() * 255)

  # display the image
  plt.imshow(img)
  plt.show()

def drawTensorImageGrayScale(tensor_image, plot=False, save=False, image_name=None):

  gray_image = np.uint8(tensor_image.squeeze().numpy() * 255)  # Assuming tensor values are in [0, 1] range

  if plot:
    img = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2RGB)

    # display the image
    plt.imshow(img)
    plt.show()

  if save:
    image = PILImage.fromarray(gray_image, mode='L')

    image.save(image_name)

def drawBoxesOnTensor(tensor_image, bboxes, plot=False, save=False, image_name=None) :

  gray_image = np.uint8(tensor_image.numpy() * 255).squeeze()  # Assuming tensor values are in [0, 1] range
  contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

  for b in bboxes:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), (0, 255, 0), 1, cv2.LINE_AA)

  # convert BGR to RGB
  img = cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)

  if plot:
    # display the image
    plt.imshow(img)
    plt.show()

  if save:
    # save the image
    cv2.imwrite(image_name, img)

def drawBoxesPredictedAndGroundTruth(tensor_image, bboxes_predicted, bboxes_groundtruth, is_normalized, plot, save, image_name):

  gray_image = np.uint8(tensor_image.numpy()).squeeze() if is_normalized else np.uint8(tensor_image.numpy() * 255).squeeze() # Assuming tensor values are in [0, 1] range
  contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
  green_color = (0, 255, 0)
  red_color   = (0, 0, 255)

  for b in bboxes_predicted:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), red_color, 1, cv2.LINE_AA)

  for b in bboxes_groundtruth:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), green_color, 1, cv2.LINE_AA)

  # convert BGR to RGB
  img = cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)

  if plot:
    # display the image
    plt.imshow(img)
    plt.show()

  if save:
    if DEBUG_FLAG:
      print(f'Saving in {image_name}')
    cv2.imwrite(image_name, img)


def immediatDrawGrayImg(gray_image):
  img = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2RGB)
  plt.imshow(img)
  plt.show()


def saveTensorToImageGrayScale(tensor_image, image_name):

  gray_image = np.uint8(tensor_image.squeeze().numpy() * 255)  # Assuming tensor values are in [0, 1] range

  image = PILImage.fromarray(gray_image, mode='L')

  image.save(image_name)


def saveBoxesOnTensorToImage(tensor_image, bboxes, image_name):
  gray_image = np.uint8(tensor_image.numpy() * 255).squeeze()  # Assuming tensor values are in [0, 1] range
  contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

  for b in bboxes:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), (0, 255, 0), 1, cv2.LINE_AA)

  # convert BGR to RGB
  img = cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)

  # save the image
  cv2.imwrite(image_name, img)


def saveBoxesPredictedAndGroundTruth(tensor_image, bboxes_predicted, bboxes_groundtruth, image_name):
  # gray_image = np.uint8(tensor_image.numpy() * 255).squeeze()  # Assuming tensor values are in [0, 1] range
  gray_image = np.uint8(tensor_image.numpy() * 255).squeeze() # Assuming tensor values are in [0, 1] range
  contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
  green_color = (0, 255, 0)
  red_color   = (255, 0, 0)

  for b in bboxes_predicted:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), red_color, 1, cv2.LINE_AA)

  for b in bboxes_groundtruth:
    x1, y1, x2, y2 = [int(elem) for elem in b]
    cv2.rectangle(contour_image, (x1, y1), (x2, y2), green_color, 1, cv2.LINE_AA)

  # cv2.drawContours(contour_image, contours, -1, (0, 0, 255), 2, cv2.LINE_AA)
  # convert BGR to RGB
  img = cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)
  cv2.imwrite(image_name, img)




def plotTrainval_losses(train_losses, val_losses):
  # Convert the loss history lists to numpy arrays
  train_loss_history = np.array(train_losses)
  val_loss_history = np.array(val_losses)

  # Plot the loss curves
  plt.plot(train_loss_history, label='Training loss')
  plt.plot(val_loss_history, label='Validation loss')
  plt.legend()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.show()



# Componentes conexas

In [15]:
def binarizarTensor(tensor_image, bin_threshold_percentaje = 0.5):
     # convertir el tensor a true/false si superan o no el umbral
    return tensor_image > bin_threshold_percentaje


def getConnectedComponents(tensor_image, bin_threshold_percentaje, min_area = 100, morphologyOps = True):
  """
    Input:
      tensor_image: tensor of values [0, 1]

    Return bounding boxes as [x1, y1, x2, y2]
  """
  # bin_threshold = int(bin_threshold_percentaje * 255)

  # Convert NumPy array to grayscale OpenCV image
  # gray_image = np.uint8(tensor_image.numpy() * 255).squeeze()  # Assuming tensor values are in [0, 1] range

  # Create a binary image
  # _, binary_image = cv2.threshold(gray_image, bin_threshold, 255, cv2.THRESH_BINARY)
  # binary_image2 = (gray_image > bin_threshold).astype(np.uint8) * 255

  # convertir el tensor a true/false si superan o no el umbral
  pixels_with_higher_umbral = tensor_image.numpy().squeeze() > bin_threshold_percentaje

  # convertir el tensor en uint8 con valores 0 o 255 en negro/blanco
  binary_image = pixels_with_higher_umbral.astype(np.uint8) * 255

  # print('BINARY IMAGE')
  # immediatDrawGrayImg(binary_image)

  if morphologyOps:
    # definir un kernel de 3x3
    kernel = np.ones((5,5), np.uint8)

    # aplicar la operación de apertura
    opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)

    # aplicar la operación de cierre
    binary_image = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # print('AFTER MORPHOLOGIC OPS')
    # immediatDrawGrayImg(binary_image)

  # Find contours in the binary image
  contours, _ =  cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # `contours` is a list of contours found in the image
  # `hierarchy` is the optional output hierarchy of the contours

  # Loop over the contours and draw them on the original image
  # contour_image = cv2.cvtColor(closing, cv2.COLOR_GRAY2BGR)
  detected_boxes = 0

  bboxes = []
  for c in contours:
      area = cv2.contourArea(c)
      if area > min_area:
        (x, y, w, h) = cv2.boundingRect(c)
        bboxes.append([x, y, x+w, y+h])
        detected_boxes += 1

  # drawBoxesPredictedAndGroundTruth(torch.from_numpy(closing), bboxes, targetBoxes, is_normalized = True)

  return bboxes



# Train functions

## Helpers

In [16]:
def getModelFileName(dataset_name: str, dropout_value: float, uses_redimension_vertical: bool, uses_redimension_horizontal: bool):
    sae_file = 'SAE'

    sae_file += f'_D{int(dropout_value * 10)}'

    if uses_redimension_vertical or uses_redimension_horizontal:
        sae_file += '_R'
        if uses_redimension_vertical:
            sae_file += 'V'
        if uses_redimension_horizontal:
            sae_file += 'H'

    return f'{sae_file}_{dataset_name}'

def getLogsTrainingTrainFolder(dropout_value: float, uses_redimension_vertical: bool, uses_redimension_horizontal: bool):
    r_folder = 'R'
    if uses_redimension_vertical:
        r_folder += 'V'
    if uses_redimension_horizontal:
        r_folder += 'H'
    return f'{DRIVE_TRAIN_LOGS_FOLDER}/trainLoss/D{dropout_value}/{r_folder}'

def getLogsTrainingValFolder(dropout_value: float, uses_redimension_vertical: bool, uses_redimension_horizontal: bool):
    r_folder = 'R'
    if uses_redimension_vertical:
        r_folder += 'V'
    if uses_redimension_horizontal:
        r_folder += 'H'
    return f'{DRIVE_TRAIN_LOGS_FOLDER}/valLoss/D{dropout_value}/{r_folder}'

def getLogsValidationFolder(val_dropout: float, times_pass_model: int, type_combination: PredictionsCombinationType):
    return f'{DRIVE_VAL_LOGS_FOLDER}/{type_combination.name}/D{val_dropout}/T{times_pass_model}'

def getLogsTestFolder(val_dropout: float, times_pass_model: int, type_combination: PredictionsCombinationType):
    return f'{DRIVE_TEST_LOGS_FOLDER}/{type_combination.name}/D{val_dropout}/T{times_pass_model}'

def getImgsValidationFolder(val_dropout: float, times_pass_model: int, type_combination: PredictionsCombinationType, dataset_name: str, model_name: str):
    return f'{DRIVE_VAL_IMG_FOLDER}/{type_combination.name}/D{val_dropout}/T{times_pass_model}/{dataset_name}/{model_name}'

def getImgsTestFolder(val_dropout: float, times_pass_model: int, type_combination: PredictionsCombinationType, dataset_name: str, model_name: str):
    return f'{DRIVE_TEST_IMG_FOLDER}/{type_combination.name}/D{val_dropout}/T{times_pass_model}/{dataset_name}/{model_name}'


def generateTrainDatasetLoader(dataset_name: str, uses_redimension_vertical: bool, uses_redimension_horizontal: bool):
    batch_size = 1

    # Create datasets to train and val
    datasetLoader = DatasetLoader(dataset_name)

    # Load the datasets
    train_json, train_img = datasetLoader.loadTrainPaths()


    dataset_train = TrainMusicDataset(
        name=dataset_name,
        jsonPaths=train_json,
        imagesPaths=train_img,
        box_resize_vertical=uses_redimension_vertical,
        box_resize_horizontal=uses_redimension_horizontal,
        resize_shape=SAE_IMAGE_SIZE,
        transforms=get_transform()
        )

    # Training dataset
    data_loader_train = DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, num_workers=0
    )

    return data_loader_train

def generateValDatasetLoaderForTrain(dataset_name, uses_redimension_vertical, uses_redimension_horizontal):
    batch_size = 1

    # Create datasets to train and val
    datasetLoader = DatasetLoader(dataset_name)

    # Load the datasets
    val_json, val_img = datasetLoader.loadValPaths()


    dataset_eval  = TrainMusicDataset(
        name=dataset_name,
        jsonPaths=val_json,
        imagesPaths=val_img,
        box_resize_vertical=uses_redimension_vertical,
        box_resize_horizontal=uses_redimension_horizontal,
        resize_shape=SAE_IMAGE_SIZE,
        transforms=get_transform()
        )


    # Validation  dataset (no redimension in order to calculate metrics)
    data_loader_eval = DataLoader(
        dataset_eval, batch_size=batch_size, shuffle=True, num_workers=0
    )

    return data_loader_eval

def generateValDatasetLoaderForTest(dataset_name):
    batch_size = 1

    # Create datasets to train and val
    datasetLoader = DatasetLoader(dataset_name)

    # Load the datasets
    val_json, val_img = datasetLoader.loadValPaths()


    dataset_eval  = TestMusicDataset(
        name=dataset_name,
        jsonPaths=val_json,
        imagesPaths=val_img,
        resize_shape=SAE_IMAGE_SIZE,
        transforms=get_transform()
        )


    # Validation  dataset (no redimension in order to calculate metrics)
    data_loader_eval = DataLoader(
        dataset_eval, batch_size=batch_size, shuffle=True, num_workers=0
    )

    return data_loader_eval


def generateTestDatasetLoader(dataset_name):
    batch_size = 1

    # Create datasets to train and val
    datasetLoader = DatasetLoader(dataset_name)

    # Load the datasets
    test_json, test_img = datasetLoader.loadTestPaths()


    dataset_test  = TestMusicDataset(
        name=dataset_name,
        jsonPaths=test_json,
        imagesPaths=test_img,
        resize_shape=SAE_IMAGE_SIZE,
        transforms=get_transform())


    # Validation  dataset (no redimension in order to calculate metrics)
    data_loader_test = DataLoader(
        dataset_test, batch_size=batch_size, shuffle=True, num_workers=0
    )

    return data_loader_test

def forwardToModel(model, image, times_pass_model, type_combination):
    if type_combination == PredictionsCombinationType.MEAN:
        result = model(image.to(device))
        for i in range(1, times_pass_model):
            print(f'\r\tForward passing with mean result {i+1}', end='')
            result += model(image.to(device))
        result /= times_pass_model
        print()
        return result.cpu()

    elif type_combination == PredictionsCombinationType.MAX:
        result = model(image.to(device))
        for i in range(1, times_pass_model):
            print(f'\r\tForward passing with max result {i+1}', end='')
            result = torch.max(result, model(image.to(device)))
        print()
        return result.cpu()

    else:
        return model(image.to(device)).cpu()



## TFM Train

Obtiene los parámetros que debe tener el modelo y entrena un modelo con esos parámetros.

In [17]:
def TFMTrainModel(dataset_name: str, dropout_value: float, 
                  uses_redimension_vertical: bool, uses_redimension_horizontal: bool, 
                  save_train_info: bool, plot_epochs: bool
                  ):
    # Crear cache
    gc.collect()
    torch.cuda.empty_cache()

    # Generate path to save model
    sae_file = getModelFileName(dataset_name=dataset_name, dropout_value=dropout_value, uses_redimension_vertical=uses_redimension_vertical, uses_redimension_horizontal=uses_redimension_horizontal)

    path_checkpoint = f'{DRIVE_MODELS_FOLDER}/{sae_file}.pt'

    train_loss_log_folder = getLogsTrainingTrainFolder (
        dropout_value=dropout_value,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )
    val_loss_log_folder = getLogsTrainingValFolder (
        dropout_value=dropout_value,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )

    train_loss_log_file = f'{train_loss_log_folder}/{sae_file}.md'
    val_loss_log_file = f'{val_loss_log_folder}/{sae_file}.md'

    print(f'Training {sae_file}')

    if pathExist(val_loss_log_file) and pathExist(train_loss_log_file):
      print('SKIPPED')
      return

    makeDirIfNotExist(DRIVE_MODELS_FOLDER)
    makeDirIfNotExist(train_loss_log_folder)
    makeDirIfNotExist(val_loss_log_folder)

    # Create model
    model = SAE()
    model.to(device)

    model.set_dropout_probability(dropout_probability=dropout_value)

    # Create datasets to train and val
    data_loader_train = generateTrainDatasetLoader (
        dataset_name=dataset_name,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )
    data_loader_eval = generateValDatasetLoaderForTrain (
        dataset_name=dataset_name,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )

    # Set losses
    loss_function = torch.nn.MSELoss()
    val_loss_function = torch.nn.MSELoss()
    train_losses = []
    val_losses = []

    # Params and optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    # optimizer = torch.optim.Adam(model.parameters(), lr = 1e-1, weight_decay = 1e-8)
    optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=1e-5, nesterov=True)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)



    # Callbacks
    modelCheckpoint = ModelCheckpoint(path_checkpoint, model)
    earlyStopping = EarlyStopping(patience=TRAIN_PATIENTE, epochs=TRAIN_NUM_EPOCHS, checkpoint=modelCheckpoint, epsilon=0.00001)

    # Num batches
    num_batches_train = len(data_loader_train)
    num_batches_val = len(data_loader_eval)


    for epoch in range(TRAIN_NUM_EPOCHS):
        if DEBUG_FLAG:
            print(f'Epoch: {epoch+1}/{TRAIN_NUM_EPOCHS}')

        ### TRAIN ONE EPOCH
        # Set model to train
        model.train()

        epochTrainLoss = []
        for iteration, batch in enumerate(data_loader_train):
            image, info, targetImage = batch

            if DEBUG_FLAG:
                print(f'\r\tTraining Batch {iteration+1} of {num_batches_train} ({info["name"]})', end='')

            # Get the inputs and labels from the batch
            image, targetImage = image.to(device), targetImage.to(device)

            # Forward pass
            reconstructed = model(image)
            loss = loss_function(reconstructed, targetImage)
            # reconstructed.detach(), image.detach(), targetImage.detach()

            # Backward pass and update parameters
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            # Append training loss
            epochTrainLoss.append(loss.item())

        if DEBUG_FLAG:
            print()

        # Optimizer step
        lr_scheduler.step()

        # clear GPU memory usage
        torch.cuda.empty_cache()
        gc.collect()

        ### VALIDATE ONE EPOCH
        # Validation
        model.eval()
        epochValLoss = []
        with torch.no_grad():
            for iteration, batch in enumerate(data_loader_eval):
                if DEBUG_FLAG:
                    print(f'\r\tValidation Batch {iteration+1} of {num_batches_val}', end='')

                # Get the inputs and labels from the batch
                image, _, targetImage = batch
                image, targetImage = image.to(device), targetImage.to(device)

                # Forward pass
                reconstructed = model(image)

                # Get loss function
                valLoss = val_loss_function(reconstructed,  targetImage)

                # Save loss for single example
                epochValLoss.append(valLoss.item())

            if DEBUG_FLAG:
                print()

        # Losses
        mean_train_loss = np.mean(epochTrainLoss)
        mean_val_loss = np.mean(epochValLoss)

        train_losses.append(mean_train_loss)
        val_losses.append(mean_val_loss)

        print(f'\tEpoch {epoch}: \t Train Loss - {round(mean_train_loss, 5)} | Val Loss - {round(mean_val_loss, 5)}')

        # Update EarlyStopping
        stopTraining = earlyStopping.update(mean_val_loss, epoch)

        # clear GPU memory usage
        torch.cuda.empty_cache()
        gc.collect()

        # If EarlyStopping says to stop, we stop training
        if stopTraining:
            print('Early Stopped!')
            break

    # Plot train val losses
    if plot_epochs:
        plotTrainval_losses(train_losses, val_losses)

    if save_train_info:
        # Save Log of the training
        with open(train_loss_log_file, 'w') as f:
            [f.write(f'{it}, {result}\n') for it, result in enumerate(train_losses)]

        with open(val_loss_log_file, 'w') as f:
            [f.write(f'{it}, {result}\n') for it, result in enumerate(val_losses)]


    return model

# Validation functions

In [24]:
def TFMValidation(
    dataset_name: str, dropout_value: float, val_dropout: float, 
    times_pass_model: int, type_combination: PredictionsCombinationType, 
    uses_redimension_vertical: bool, uses_redimension_horizontal: bool, 
    save_val_info: bool, save_val_imgs: bool
    ):
    # Crear cache
    gc.collect()
    torch.cuda.empty_cache()

    # Generate path of the saved model
    sae_file = getModelFileName (
        dataset_name=dataset_name,
        dropout_value=dropout_value,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )

    # Generate folder to save this validation process
    folder_validation = getLogsValidationFolder (
        val_dropout=val_dropout,
        times_pass_model=times_pass_model,
        type_combination=type_combination
        )

    makeDirIfNotExist(folder_validation)

    path_model = f'{DRIVE_MODELS_FOLDER}/{sae_file}.pt'

    val_best_bin_log_folder = f'{folder_validation}/bestBin'
    val_f1_log_folder = f'{folder_validation}/valF1'
    val_iou_log_folder = f'{folder_validation}/valIoU'

    val_best_bin_log_file = f'{val_best_bin_log_folder}/{sae_file}.bestBin'
    val_f1_log_file = f'{val_f1_log_folder}/{sae_file}.valF1'
    val_iou_log_file = f'{val_iou_log_folder}/{sae_file}.valIoU'

    makeDirIfNotExist(val_best_bin_log_folder)
    makeDirIfNotExist(val_f1_log_folder)
    makeDirIfNotExist(val_iou_log_folder)

    val_img_folder = getImgsValidationFolder (
        val_dropout=val_dropout,
        times_pass_model=times_pass_model,
        type_combination=type_combination,
        dataset_name=dataset_name,
        model_name=sae_file
        )

    print(f'Evaluating {sae_file}')

    # Create model
    model: SAE = torch.load(path_model, map_location=torch.device(device))
    model.to(device)

    model.eval()
    if val_dropout > 0:
        model.enable_eval_dropout()
        model.set_dropout_probability(dropout_probability=val_dropout)

    # Create datasets to train and val
    data_loader_eval = generateValDatasetLoaderForTest (
        dataset_name=dataset_name
        )


    # Evaluation
    bin_F1score_map = {i: 0.0 for i in BIN_UMBRALS}
    bin_IoUscore_map = {i: 0.0 for i in BIN_UMBRALS}

    with torch.no_grad():
        # Iterate over each example of the eval dataset
        for iteration, batch in enumerate(data_loader_eval):
            if DEBUG_FLAG:
                print(f'Eval with batch {iteration}/{len(data_loader_eval)}')

            # Get the inputs and labels from the batch
            image, target = batch
            targetBoxes = target.squeeze().numpy().tolist()

            # Forward pass
            result = forwardToModel(model=model,
                                    image=image,
                                    times_pass_model=times_pass_model,
                                    type_combination=type_combination
                                    )

            # Creates a dictionary with the bin thresholds and F1 score for each
            for bin_umbral in BIN_UMBRALS:
                if DEBUG_FLAG:
                    print(f'\r\tTesting with {bin_umbral} binarization umbral', end='')

                # Extract BB from prediction
                boxes = getConnectedComponents(result, bin_threshold_percentaje=bin_umbral)
                if uses_redimension_vertical or uses_redimension_horizontal:  # If we're using a resized BB, resize it to original
                    vResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_vertical   else 1
                    hResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_horizontal else 1
                    boxes = [resize_box(box, vResize=vResize, hResize=hResize) for box in boxes]

                # Calculate F1 and IoU
                f1, matched_ious = calculate_F1(y_true = targetBoxes, y_pred = boxes, iou_threshold=0.5)

                # Accumulate F1 and IoU score in the bin umbral used for this concrete example
                bin_F1score_map[bin_umbral]  += f1
                bin_IoUscore_map[bin_umbral] += np.mean(matched_ious) if len(matched_ious) > 0 else 0

            if DEBUG_FLAG:
                print()

            torch.cuda.empty_cache()
            # gc.collect()


    # Calculate mean of F1 and IoU scores
    number_elements = len(data_loader_eval)
    for key in bin_F1score_map:
        bin_F1score_map[key] /= number_elements

    for key in bin_IoUscore_map:
        bin_IoUscore_map[key] /= number_elements

    # Calculate mean of each bin theshold and decide max
    best_bin_threshold = max(bin_F1score_map, key=bin_F1score_map.get)

    # Check for dupes in best F1 score
    common_keys = []
    for key, value in bin_F1score_map.items():
        if value == bin_F1score_map[best_bin_threshold]:
            common_keys.append(key)

    # If there's dupes, select the one with most IoU
    if len(common_keys) > 1:
        for k in common_keys:
            best_bin_threshold = best_bin_threshold if bin_IoUscore_map[best_bin_threshold] > bin_IoUscore_map[k] else k


    # Create the "best umbral" info message
    stringBestMsg = f'Evaluated {sae_file}: \t Bin Threshold {best_bin_threshold} --> F1 - {bin_F1score_map[best_bin_threshold]} | IoU - {bin_IoUscore_map[best_bin_threshold]}'
    print(stringBestMsg)

    # Save in a file the F1 and IoU metrics
    if save_val_info:
        with open(val_best_bin_log_file, 'w') as f:
            f.write(f'{best_bin_threshold}\n{stringBestMsg}')

        stringF1List = '\n'.join([f' {round(key, 2)} {value}' for key, value in bin_F1score_map.items()])
        with open(val_f1_log_file, 'w') as f:
            f.write(stringF1List)

        stringIoUList = '\n'.join([f' {round(key, 2)} {value}' for key, value in bin_IoUscore_map.items()])
        with open(val_iou_log_file, 'w') as f:
            f.write(stringIoUList)

    return best_bin_threshold

## TFM Forward

In [ ]:
def getPredictionModel(dataset_name):
  model = torch.load(f'SAE_{dataset_name}.pt', map_location=torch.device(device))
  model.to(device)
  return model

def TFMForward(
    dataset_name, image: PILImage, uses_redimension_vertical: bool, uses_redimension_horizontal: bool, 
    bin_umbral: float, val_dropout: float, times_pass_model: int, type_combination: PredictionsCombinationType
    ):
    """Forward de una imagen para obtener los bounding boxes

    Args:
        dataset_name (string): Nombre del dataset
        image (Image): Imagen a hacer forward
        uses_redimension_vertical (bool): Si el modelo usa redimensión vertical
        uses_redimension_horizontal (bool): Si el modelo usa redimensión horizontal
        bin_umbral (float): Umbral de binarización
        val_dropout (float): Dropout a aplicar en prediccion
        times_pass_model (int): Cantidad de predicciones a combinar en prediccion
        type_combination (PredictionsCombinationType): Tipo de combinacion a aplicar sobre las predicciones

    Returns:
        list: Bounding boxes extraídas de la imagen
    """
    # Create model
    model = getPredictionModel(dataset_name=dataset_name)

    if val_dropout > 0:
        model.enable_eval_dropout()
        model.set_dropout_probability(dropout_probability=val_dropout)
    
    image = image.resize(SAE_IMAGE_SIZE)
    
    transforms = get_transform()
    transformedImage = transforms(image)
    
    with T.no_grad():
        result = forwardToModel(model=model,
                                image=transformedImage,
                                times_pass_model=times_pass_model,
                                type_combination=type_combination
                                )

        boxes = getConnectedComponents(result, bin_threshold_percentaje=bin_umbral)

        if uses_redimension_vertical or uses_redimension_horizontal:
            vResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_vertical   else 1
            hResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_horizontal else 1
            boxes = [resize_box(box, vResize=vResize, hResize=hResize) for box in boxes]

    return boxes

# Test functions

In [19]:
def TFMTest(
    dataset_name: str, dropout_value: float, val_dropout: float, 
    times_pass_model: int, type_combination: PredictionsCombinationType,
    uses_redimension_vertical: float, uses_redimension_horizontal: float, 
    save_test_info: bool, save_test_img: bool
    ):
    # Crear cache
    gc.collect()
    torch.cuda.empty_cache()

    # Generate path of the saved model
    sae_file = getModelFileName (
        dataset_name=dataset_name,
        dropout_value=dropout_value,
        uses_redimension_vertical=uses_redimension_vertical,
        uses_redimension_horizontal=uses_redimension_horizontal
        )

    # Get folder to obtain best bin threshold
    folder_validation = getLogsValidationFolder (
        val_dropout=val_dropout,
        times_pass_model=times_pass_model,
        type_combination=type_combination
        )

    val_best_bin_log_file = f'{folder_validation}/bestBin/{sae_file}.bestBin'
    bin_umbral_for_model = 0
    with open(val_best_bin_log_file, 'r') as file:
        bin_umbral_for_model = float(file.readline())

    folder_test = getLogsTestFolder (
        val_dropout=val_dropout,
        times_pass_model=times_pass_model,
        type_combination=type_combination
        )

    test_img_folder = getImgsValidationFolder (
        val_dropout=val_dropout,
        times_pass_model=times_pass_model,
        type_combination=type_combination,
        dataset_name=dataset_name,
        model_name=sae_file
        )

    print(f'Testing model {sae_file} with Bin umbral {bin_umbral_for_model}')

    makeDirIfNotExist(folder_test)
    makeDirIfNotExist(test_img_folder)

    path_model = f'{DRIVE_MODELS_FOLDER}/{sae_file}.pt'

    test_best_bin_log_file = f'{folder_test}/{sae_file}.test'

    # Create model
    model: SAE = torch.load(path_model, map_location=torch.device(device))
    model.to(device)
    model.eval()

    if val_dropout > 0:
        model.enable_eval_dropout()
        model.set_dropout_probability(dropout_probability=val_dropout)

    # Create datasets to train and val
    data_loader_test = generateTestDatasetLoader (
        dataset_name=dataset_name
        )


    # Evaluation
    bin_F1score_sum = 0
    bin_IoUscore_sum = 0

    with torch.no_grad():
        # Iterate over each example of the eval dataset
        for iteration, batch in enumerate(data_loader_test):
            if DEBUG_FLAG:
                print(f'Eval with batch {iteration}/{len(data_loader_test)}')

            # Get the inputs and labels from the batch
            image, target = batch
            targetBoxes = target.squeeze().numpy().tolist()

            # Forward pass
            result = forwardToModel(model=model,
                                    image=image,
                                    times_pass_model=times_pass_model,
                                    type_combination=type_combination
                                    )

            # Creates a dictionary with the bin thresholds and F1 score for each
            if DEBUG_FLAG:
                print(f'\r\tTesting with {bin_umbral_for_model} binarization umbral', end='')

            # Extract BB from prediction
            boxes = getConnectedComponents(result, bin_threshold_percentaje=bin_umbral_for_model)
            if uses_redimension_vertical or uses_redimension_horizontal:  # If we're using a resized BB, resize it to original
                vResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_vertical   else 1
                hResize = BBOX_REDIMENSIONED_RECOVER if uses_redimension_horizontal else 1
                boxes = [resize_box(box, vResize=vResize, hResize=hResize) for box in boxes]

            # Calculate F1 and IoU
            f1, matched_ious = calculate_F1(y_true=targetBoxes, y_pred=boxes, iou_threshold=0.5)

            # Accumulate F1 and IoU score in the bin umbral used for this concrete example
            bin_F1score_sum  += f1
            bin_IoUscore_sum += np.mean(matched_ious) if len(matched_ious) > 0 else 0

            if save_test_img:
                drawBoxesPredictedAndGroundTruth (
                    tensor_image=image,
                    bboxes_predicted=boxes,
                    bboxes_groundtruth=targetBoxes,
                    is_normalized=False,
                    image_name=f'{test_img_folder}/{iteration}.png',
                    plot=False,
                    save=save_test_img
                    )

            if DEBUG_FLAG:
                print()

            torch.cuda.empty_cache()
            # gc.collect()


    # Calculate mean of F1 and IoU scores
    number_elements = len(data_loader_test)

    bin_F1score_sum /= number_elements
    bin_IoUscore_sum /= number_elements

    print(f'Tested {sae_file}: \t Bin Umbral {bin_umbral_for_model}, Combination {type_combination.value}, Times {times_pass_model} --> F1 - {bin_F1score_sum} | IoU - {bin_IoUscore_sum}')

    # Save in a file the F1 and IoU metrics
    if save_test_info:
        with open(test_best_bin_log_file, 'w') as f:
            f.write(f'F1: {bin_F1score_sum}\nIoU: {bin_IoUscore_sum}\n Test model {sae_file} with Bin Umbral {bin_umbral_for_model}, Combination {type_combination.value}, Times {times_pass_model}')



# TFM

## Paso 1

Recreamos el SAE sin Dropout.

## Paso 2

Probar redimensión de pentagramas con los 3 datasets.

Entrenamos 4 modelos sin Dropout: uno sin redimensión, otro con redimensión vertical, otro con redimensión horizontal y otro con redimensión vertical y horizontal. Entrenamos para los 3 Datasets.

In [22]:
# Models parameters
dropout_value = 0
save_train_info = True
plot_epochs = False

                                # Mod1,  Mod2,  Mod3,  Mod4
usesRedimensionVerticalList   = [False, False, True , True]
usesRedimensionHorizontalList = [False, True , False, True]

for uses_redimension_vertical, uses_redimension_horizontal in zip(usesRedimensionVerticalList, usesRedimensionHorizontalList):
    for dataset_name in DATASETS:
        TFMTrainModel(dataset_name=dataset_name,
                      dropout_value=dropout_value,
                      uses_redimension_vertical=uses_redimension_vertical,
                      uses_redimension_horizontal=uses_redimension_horizontal,
                      save_train_info=save_train_info,
                      plot_epochs=plot_epochs
                      )

Training SAE_D0_Capitan


OSError: [Errno 30] Read-only file system: '/content'

Extraemos el mejor umbral de binarización para cada modelo entrenado y obtenemos los resultados con el conjunto de validación. Fijamos la redimensión al mejor obtenido.

In [25]:
# Models parameters
dropout_value = 0
save_val_info = True
save_val_imgs = True
val_dropout = 0
times_pass_model = 1
type_combination = PredictionsCombinationType.NONE

                                # Mod1,  Mod2,  Mod3,  Mod4
usesRedimensionVerticalList   = [False, False, True , True]
usesRedimensionHorizontalList = [False, True , False, True]

for uses_redimension_vertical, uses_redimension_horizontal in zip(usesRedimensionVerticalList, usesRedimensionHorizontalList):
    for dataset_name in DATASETS:
        TFMValidation(dataset_name=dataset_name,
                      dropout_value=dropout_value,
                      val_dropout=val_dropout,
                      times_pass_model=times_pass_model,
                      type_combination=type_combination,
                      uses_redimension_vertical=uses_redimension_vertical,
                      uses_redimension_horizontal=uses_redimension_horizontal,
                      save_val_info=save_val_info,
                      save_val_imgs=save_val_imgs
                      )


Evaluating SAE_D0_Capitan


/var/folders/rx/htv12nn55bq6w7t016qbx0d80000gn/T/ipykernel_35230/2279567646.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path_model, map_location=

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/TFM/IA/models/SAE_D0_Capitan.pt'

Con el test

In [ ]:
# Models parameters
dropout_value = 0
save_test_info = True
save_test_img = True
val_dropout = 0
times_pass_model = 1
type_combination = PredictionsCombinationType.NONE

                                # Mod1,  Mod2,  Mod3,  Mod4
usesRedimensionVerticalList   = [False, False, True , True]
usesRedimensionHorizontalList = [False, True , False, True]

for uses_redimension_vertical, uses_redimension_horizontal in zip(usesRedimensionVerticalList, usesRedimensionHorizontalList):
    for dataset_name in DATASETS:
        TFMTest(dataset_name=dataset_name,
                      dropout_value=dropout_value,
                      val_dropout=val_dropout,
                      times_pass_model=times_pass_model,
                      type_combination=type_combination,
                      uses_redimension_vertical=uses_redimension_vertical,
                      uses_redimension_horizontal=uses_redimension_horizontal,
                      save_test_info=save_test_info,
                      save_test_img=save_test_img
                      )


## Paso 3

Probar Dropout en entrenamiento.

Entrenamos 6 modelos, usando Dropout desde 0 hasta 0'5, en entrenamiento. Entrenamos para los 3 Datasets.

In [ ]:
# Models parameters
save_train_info = True
plot_epochs = False

uses_redimension_vertical = True
uses_redimension_horizontal = True

for dropout_value in DROPOUT_VALUES:
    for dataset_name in DATASETS:
        TFMTrainModel(dataset_name=dataset_name,
                      dropout_value=dropout_value,
                      uses_redimension_vertical=uses_redimension_vertical,
                      uses_redimension_horizontal=uses_redimension_horizontal,
                      save_train_info=save_train_info,
                      plot_epochs=plot_epochs
                      )

Training SAE_D0_RVH_Capitan
SKIPPED
Training SAE_D0_RVH_SEILS
SKIPPED
Training SAE_D0_RVH_FMT_C
SKIPPED
Training SAE_D1_RVH_Capitan
SKIPPED
Training SAE_D1_RVH_SEILS
SKIPPED
Training SAE_D1_RVH_FMT_C
	Epoch 0: 	 Train Loss - 0.09089 | Val Loss - 0.10445
	Early stopping: [1/20] on epoch 0/100
	Epoch 1: 	 Train Loss - 0.066 | Val Loss - 0.06128
	Early stopping: [1/20] on epoch 1/100
	Epoch 2: 	 Train Loss - 0.05973 | Val Loss - 0.06309
	Early stopping: [2/20] on epoch 2/100
	Epoch 3: 	 Train Loss - 0.05557 | Val Loss - 0.05959
	Early stopping: [1/20] on epoch 3/100
	Epoch 4: 	 Train Loss - 0.05419 | Val Loss - 0.05746
	Early stopping: [1/20] on epoch 4/100
	Epoch 5: 	 Train Loss - 0.05367 | Val Loss - 0.05733
	Early stopping: [1/20] on epoch 5/100
	Epoch 6: 	 Train Loss - 0.05304 | Val Loss - 0.05731
	Early stopping: [1/20] on epoch 6/100
	Epoch 7: 	 Train Loss - 0.05178 | Val Loss - 0.05755
	Early stopping: [2/20] on epoch 7/100
	Epoch 8: 	 Train Loss - 0.05253 | Val Loss - 0.05756
	Ear

Extraemos el mejor umbral de binarización para cada modelo entrenado y obtenemos los resultados con el conjunto de validación. Fijamos el Dropout en entrenamiento al mejor obtenido.

In [ ]:
# Models parameters
save_val_info = True
val_dropout = 0
times_pass_model = 1
type_combination = PredictionsCombinationType.NONE

uses_redimension_vertical = True
uses_redimension_horizontal = True

for dropout_value in DROPOUT_VALUES:
    for dataset_name in DATASETS:
        TFMValidation(dataset_name=dataset_name,
                      dropout_value=dropout_value,
                      val_dropout=val_dropout,
                      times_pass_model=times_pass_model,
                      type_combination=type_combination,
                      uses_redimension_vertical=uses_redimension_vertical,
                      uses_redimension_horizontal=uses_redimension_horizontal,
                      save_val_info=save_val_info
                      )


## Paso 4

Probamos el Dropout en test.

# Pruebas TFM

In [ ]:
# Pruebas TFM

targetBoxes = [[0, 0, 5, 5]]
boxes = [[5, 5, 11, 15]]

print(calculate_F1(y_true = targetBoxes, y_pred = boxes, iou_threshold=0.5))

Matrix of IoU:
[[0.]]
Precision: 0.0 	 Recall: 0.0
(0, [])


# Experimento 1

## Iniciar variables y datasets

In [ ]:
dataset_name = 'Capitan' # 'Capitan'
batch_size = 1
modelName = 'SAE'
resized_shape = SAE_IMAGE_SIZE

resize_boxes = True




In [ ]:
datasetLoader = DatasetLoader(dataset_name)

# datasetLoader.drawBoxesInDataset()

# Load the datasets
# train, val, test = datasetLoader.loadAllDataset()
train_json, train_img = datasetLoader.loadTrainPaths()
val_json, val_img = datasetLoader.loadValPaths()
test_json, test_img = datasetLoader.loadTestPaths()

# train_json, train_img = train_json[:5], train_img[:5]


dataset_train = TrainMusicDataset(name=dataset_name, jsonPaths=train_json, imagesPaths=train_img, resize_shape=resized_shape, transforms=get_transform(), box_resize_vertical = False, box_resize_horizontal = False)
dataset_eval = TrainMusicDataset(name=dataset_name, jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize_vertical = False, box_resize_horizontal = False)

# Training dataset
data_loader_train = DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, num_workers=0
)

# Validation  dataset (no redimension in order to calculate metrics)
data_loader_eval = DataLoader(
    dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0
)

# Resized for coco (to check if metrics change)
dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_test = DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0
)

In [ ]:
print(f'Training on {device}')

## Save model

In [ ]:
torch.save(model.state_dict(), 'model.pth')


## Load model

In [ ]:
model_path = '/content/SAE_dropout02_redim8.pt'
# model_path = '/content/SAE_dropout02.pt'
model = torch.load(model_path, map_location=torch.device(device))
model.to(device)
# model.set_dropout_probability(0.2)
# model.load_state_dict(torch.load('model.pth', map_location=torch.device(device)))


## Training

In [ ]:
path_checkpoint = 'SAE_dropout02_redim8.pt'
model = SAE()
model.set_dropout_probability(0.2)
model = trainModel(data_loader_train, data_loader_eval,
           model = model, num_epochs = TRAIN_NUM_EPOCHS,
           train_patience = TRAIN_PATIENTE, path_checkpoint = path_checkpoint)

path_checkpoint = 'SAE_dropout03_redim8.pt'
model = SAE()
model.set_dropout_probability(0.3)
model = trainModel(data_loader_train, data_loader_eval,
           model = model, num_epochs = TRAIN_NUM_EPOCHS,
           train_patience = TRAIN_PATIENTE, path_checkpoint = path_checkpoint)

path_checkpoint = 'SAE_dropout05_redim8.pt'
model = SAE()
model.set_dropout_probability(0.5)
model = trainModel(data_loader_train, data_loader_eval,
           model = model, num_epochs = TRAIN_NUM_EPOCHS,
           train_patience = TRAIN_PATIENTE, path_checkpoint = path_checkpoint)



## Val binarization hiperparameter ajustment

In [ ]:
model_path = '/content/models/SAE_dropout02_redim8.pt'
# model_path = '/content/SAE_dropout02.pt'
model = torch.load(model_path, map_location=torch.device(device))

model.eval()
model.enable_eval_dropout()
model.to(device)

bin_umbrals_to_try = np.arange(0, 1, 0.01)
best_bin_thresholdMean = []
best_bin_thresholdMax = []


for times_pass_model in [10, 20, 30, 50, 100]:
  best_bin_thresholdMean.append(getBestIoUThreshold(model, data_loader_eval = data_loader_eval, dropout_value = 0.2, type_combination = PredictionsCombinationType.MEAN, model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model))

for times_pass_model in [10, 20, 30, 50, 100]:
  best_bin_thresholdMax.append(getBestIoUThreshold(model, data_loader_eval = data_loader_eval, type_combination = 'max', model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model))

best_bin_threshold = max(max(best_bin_thresholdMean), max(best_bin_thresholdMax))

In [ ]:
# Validation  dataset (no redimension in order to calculate metrics)
dataset_eval = TrainMusicDataset(name=dataset_name, jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_eval = DataLoader(
    dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0
)



# Model 20% Dropout resized
model_path = '/content/models/SAE_dropout02_redim8.pt'
model_dropout = 0.2
model = torch.load(model_path, map_location=torch.device(device))

model.eval()
model.to(device)

bin_umbrals_to_try = np.arange(0, 1, 0.01)
best_bin_thresholdMean = []
best_bin_thresholdMax = []

times_pass_model = 1

best_bin_thresholdMean.append(getBestIoUThreshold(model, type_combination = 'max', model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model, dropout_value=model_dropout))


model_path = '/content/models/SAE_dropout03_redim8.pt'
model_dropout = 0.3
model = torch.load(model_path, map_location=torch.device(device))

model.eval()
model.to(device)

bin_umbrals_to_try = np.arange(0, 1, 0.01)
best_bin_thresholdMean = []
best_bin_thresholdMax = []


best_bin_thresholdMax.append(getBestIoUThreshold(model, type_combination = 'max', model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model, dropout_value=model_dropout))

model_path = '/content/models/SAE_dropout05_redim8.pt'
model_dropout = 0.5
model = torch.load(model_path, map_location=torch.device(device))

model.eval()
model.to(device)

bin_umbrals_to_try = np.arange(0, 1, 0.01)
best_bin_thresholdMean = []
best_bin_thresholdMax = []


best_bin_thresholdMax.append(getBestIoUThreshold(model, type_combination = 'max', model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model, dropout_value=model_dropout))



In [ ]:
import shutil
import os


# Path to the source folder
source_folder = '/content'

# Path to the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/TFG/BinResults'

# Get a list of all files in the source folder
file_list = os.listdir(source_folder)

# Loop over each file in the list
for file_name in file_list:
    # Check if the file name starts with "IoU" or "F1"
    if file_name.startswith('IoU') or file_name.startswith('F1'):
        # Construct the full path to the file
        file_path = os.path.join(source_folder, file_name)

        # Move the file to the destination folder in Google Drive
        shutil.copy(file_path, destination_folder)


## Test binarization thresholds

In [ ]:
modelNames = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5', 'Model6']
modelLoad  = {
    modelNames[0]: 'SAE_dropout03.pt',
    modelNames[1]: 'SAE_dropout02_redim8.pt',
    modelNames[2]: 'SAE_dropout02.pt',
    modelNames[3]: 'SAE_dropout03.pt',
    modelNames[4]: 'SAE_dropout02_redim8.pt',
    modelNames[5]: 'SAE_dropout02_redim8.pt',
}

modelResizedBB  = {
    modelNames[0]: False,
    modelNames[1]: True,
    modelNames[2]: False,
    modelNames[3]: False,
    modelNames[4]: True,
    modelNames[5]: True,
}

modelPredictDropout  = {
    modelNames[0]: False,
    modelNames[1]: False,
    modelNames[2]: True,
    modelNames[3]: True,
    modelNames[4]: True,
    modelNames[5]: True,
}

modelDropoutProb = {
    modelNames[0]: 0.3,
    modelNames[1]: 0.2,
    modelNames[2]: 0.2,
    modelNames[3]: 0.3,
    modelNames[4]: 0.2,
    modelNames[5]: 0.2,
}

modelCombinationOp  = {
    modelNames[0]: 'max',
    modelNames[1]: 'max',
    modelNames[2]: 'max',
    modelNames[3]: 'mean',
    modelNames[4]: 'max',
    modelNames[5]: 'mean',
}

modelCombinationNum  = {
    modelNames[0]: 1,
    modelNames[1]: 1,
    modelNames[2]: 10,
    modelNames[3]: 100,
    modelNames[4]: 10,
    modelNames[5]: 100,
}

modelBestUmbral  = {
    modelNames[0]: 0.75,
    modelNames[1]: 0.65,
    modelNames[2]: 0.97,
    modelNames[3]: 0.61,
    modelNames[4]: 0.88,
    modelNames[5]: 0.41,
}

In [ ]:
modelNames = [ 'Model5', 'Model6']
modelLoad  = {
    modelNames[0]: 'SAE_dropout02_redim8.pt',
    modelNames[1]: 'SAE_dropout02_redim8.pt',
}

modelResizedBB  = {
    modelNames[0]: True,
    modelNames[1]: True,
}

modelPredictDropout  = {
    modelNames[0]: True,
    modelNames[1]: True,
}

modelDropoutProb = {
    modelNames[0]: 0.2,
    modelNames[1]: 0.2,
}

modelCombinationOp  = {
    modelNames[0]: 'max',
    modelNames[1]: 'mean',
}

modelBestUmbral  = {
    modelNames[0]: 0.88,
    modelNames[1]: 0.41,
}

In [ ]:
dirToSave = '/content/drive/MyDrive/TFG/testResultsCapitan'

for modelName in modelNames:
    model_path = f'/content/models/exp1/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)
    # for times_pass_model in [10, 20, 30, 50, 100]:
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  = modelCombinationNum[modelName],
              type_combination  =  modelCombinationOp[modelName])



In [ ]:
### Test multiple passes

dirToSave = '/content/drive/MyDrive/TFG/testResultsCapitan'

for modelName in modelNames:
    model_path = f'/content/models/exp1/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)
    for times_pass_model in [10, 20, 30, 50, 100]:
        testModel(model,
                  model_name = modelName,
                  dir_to_save = dirToSave,
                  data_loader_test = data_loader_test,
                  bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
                  model_resize_boxes = modelResizedBB[modelName],
                  times_pass_model  = times_pass_model,
                  type_combination  =  modelCombinationOp[modelName])



In [ ]:
#Save images

modelName = 'Model6'
model_path = f'/content/models/exp1/{modelLoad[modelName]}'
model = torch.load(model_path, map_location=torch.device(device))
print(f'Loading model {modelLoad[modelName]}')
model.eval()

if modelPredictDropout[modelName]:
    model.enable_eval_dropout()
    model.set_dropout_probability(modelDropoutProb[modelName])

model.to(device)
for times_pass_model in [10, 20, 30, 50, 100]:
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  = times_pass_model,
              type_combination  =  modelCombinationOp[modelName])


## Test

In [ ]:
datasetLoader = DatasetLoader(dataset_name)
test_json, test_img = datasetLoader.loadTestPaths()

resize_boxes = True

if resize_boxes:
  dataset_testing = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
else:
  dataset_testing = TestMusicDataset(name=dataset_name, jsonPaths=train_json, imagesPaths=train_img, resize_shape=resized_shape, transforms=get_transform())

# Training dataset
data_loader_testing = DataLoader(
    dataset_testing, batch_size=batch_size, shuffle=True, num_workers=0
)




model_path = '/content/models/exp1/SAE_dropout02.pt'
# model_path = '/content/SAE_dropout02.pt'
model = torch.load(model_path, map_location=torch.device(device))
model.eval()
model.enable_eval_dropout()
model.set_dropout_probability(0.2)


times_pass_model =  100
type_combination = PredictionsCombinationType.MEAN



# Evaluation
f1_metric = []
for iteration, batch in enumerate(data_loader_testing):
    print()
    print(iteration)
    print()

    # Get the inputs and labels from the batch
    image, target = batch
    # image, targetImage = image.to(device), targetImage.to(device)

    model.to(device)
    # Forward pass
    reconstructed = forwardToModel(model=model,
                                    image=image,
                                    times_pass_model=times_pass_model,
                                    type_combination=type_combination
                                    )
    detached_reconstructed = reconstructed.cpu().detach()
    boxes = getConnectedComponents(detached_reconstructed, bin_threshold_percentaje=0.41)
    boxes = [resize_box(box, BBOX_REDIMENSIONED_RECOVER, BBOX_REDIMENSIONED_RECOVER) for box in boxes]
    saveBoxesOnTensorToImage(tensor_image=image, bboxes=boxes, image_name=f'/content/drive/MyDrive/TFG/images/{target["name"]}_bestModel_bin0.41.png')


    reconstructed = forwardToModel(model=model,
                                    image=image,
                                    times_pass_model=times_pass_model,
                                    type_combination=type_combination
                                    )
    detached_reconstructed = reconstructed.cpu().detach()
    boxes = getConnectedComponents(detached_reconstructed, bin_threshold_percentaje=0.7)
    saveBoxesOnTensorToImage(tensor_image=image, bboxes=boxes, image_name=f'/content/drive/MyDrive/TFG/images/{target["name"]}_bestModel_bin0.7.png')

    # f1, matched_ious = calculate_F1(y_true = targetBoxes, y_pred = boxes, iou_threshold=0.5)
    # f1_metric.append(f1)

    # print(f1)
    # drawBoxesPredictedAndGroundTruth(detached_reconstructed, boxes, targetBoxes)
    # drawBoxesOnTensor(reconstructed.cpu().detach(), boxes)

    # targetBoxes = target['boxes'].squeeze().numpy().tolist()
    # saveBoxesOnTensorToImage(tensor_image=image, bboxes=targetBoxes, image_name=f'/content/drive/MyDrive/TFG/images/{target["name"]}_GT.png')
    # saveTensorToImageGrayScale(tensor_image=image, image_name=f'/content/drive/MyDrive/TFG/images/{iteration}_GT.png')
    # drawBoxesOnTensor(reconstructed.cpu().detach(),targetBoxes)

    torch.cuda.empty_cache()
    gc.collect()
    print()

# print(np.mean(f1_metric))




# drawBoxesOnTensor(reconstructed.cpu().detach(), boxes)
# drawBoxesOnTensor(reconstructed.cpu().detach(),targetBoxes)

## One image eval

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    model.to(device)

    evalData = iter(data_loader_eval)
    next(evalData)
    next(evalData)
    batch = next(evalData)
    image, target, targetImage = batch

    GT_bbox = target['boxes'].squeeze().numpy().tolist()

    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass1.jpg')
    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass2.jpg')
    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass3.jpg')

    # drawTensorImageGrayScale(model(image.to(device)).cpu())
    # drawTensorImageGrayScale(model(image.to(device)).cpu())
    # drawTensorImageGrayScale(model(image.to(device)).cpu())

    imagenRes = model(image.to(device)).cpu()
    imagen_bin5 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.3)
    imagen_bin7 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.5)
    imagen_bin9 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.7)

    # saveTensorToImageGrayScale(imagenRes, 'bin_prediction.jpg')
    # saveTensorToImageGrayScale(imagen_bin5, 'bin_output3.jpg')
    # saveTensorToImageGrayScale(imagen_bin7, 'bin_output5.jpg')
    # saveTensorToImageGrayScale(imagen_bin9, 'bin_output7.jpg')

    # saveTensorToImageGrayScale(imagenRes, 'bin_prediction.jpg')
    drawTensorImageGrayScale(imagen_bin5)
    drawTensorImageGrayScale(imagen_bin7)
    # saveTensorToImageGrayScale(imagen_bin9, 'bin_output7.jpg')


    # print('Mean result of 10 passes')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'mean'), 'dropout_10mean_result.jpg')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'max'), 'dropout_10max_result.jpg')

    bboxes = getConnectedComponents(imagen_bin7)

    drawBoxesOnTensor(imagen_bin7, bboxes)
    saveBoxesOnTensorToImage(imagen_bin7, bboxes, 'bin_boxes5.jpg')

    resizedBboxes = [resize_box(box, BBOX_REDIMENSIONED_RECOVER, BBOX_REDIMENSIONED_RECOVER) for box in bboxes]

    saveBoxesPredictedAndGroundTruth(imagen_bin7, resizedBboxes, bboxes, 'resized_model_output.jpg')

    saveBoxesPredictedAndGroundTruth(image, resizedBboxes, GT_bbox, 'resized_model_GT.jpg')
    # drawTensorImageGrayScale(targetImage)
    # print(target['boxes'])

## Testing Resize BB

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    model.to(device)

    evalData = iter(data_loader_train)
    next(evalData)
    next(evalData)
    batch = next(evalData)
    image, target, targetImage = batch

    boxes = [box for box in target['boxes'].squeeze().numpy().tolist()]
    boxesResized = [resize_box(box, 1/0.8,  1/0.8) for box in boxes]

    print(boxes)
    print(boxesResized)

    imageDraw = draw_back_white_bb_image(height=512, width=512, boxes=boxes)
    imageDrawResized = draw_back_white_bb_image(height=512, width=512, boxes=boxesResized)

    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass1.jpg')
    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass2.jpg')
    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass3.jpg')

    # drawTensorImageGrayScale(model(image.to(device)).cpu())
    # drawTensorImageGrayScale(model(image.to(device)).cpu())
    # drawTensorImageGrayScale(model(image.to(device)).cpu())

    # imagenRes = model(image.to(device)).cpu()
    # imagen_bin5 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.3)
    # imagen_bin7 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.5)
    # imagen_bin9 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.7)

    # saveTensorToImageGrayScale(imagenRes, 'bin_prediction.jpg')
    # saveTensorToImageGrayScale(imagen_bin5, 'bin_output3.jpg')
    # saveTensorToImageGrayScale(imagen_bin7, 'bin_output5.jpg')
    # saveTensorToImageGrayScale(imagen_bin9, 'bin_output7.jpg')


    # Convert the PIL image to a NumPy array

    img_array = np.array(imageDraw)

    # Plot the image using matplotlib
    plt.imshow(img_array)
    plt.show()

    drawTensorImageGrayScale(image[0][0])
    drawTensorImageGrayScale(targetImage[0][0])
    # saveTensorToImageGrayScale(imagenRes, 'bin_prediction.jpg')
    # drawTensorImageGrayScale(imagen_bin7)
    # saveTensorToImageGrayScale(imagen_bin9, 'bin_output7.jpg')


    # print('Mean result of 10 passes')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'mean'), 'dropout_10mean_result.jpg')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'max'), 'dropout_10max_result.jpg')

    # bboxes = getConnectedComponents(imagen_bin7)

    # drawBoxesOnTensor(imagen_bin7, bboxes)
    # saveBoxesOnTensorToImage(imagen_bin7, bboxes, 'bin_boxes5.jpg')
    # drawTensorImageGrayScale(targetImage)
    # print(target['boxes'])

# Experimento 2

## Iniciar variables y datasets

In [ ]:
dataset_name = 'Capitan' # 'Capitan'
batch_size = 1
modelName = 'SAE'
resized_shape = SAE_IMAGE_SIZE
path_checkpoint = modelName.lower() + "_save"

resize_boxes = True

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Train

In [ ]:
modelNames = ['Model1']

modelSavePath  = {
    modelNames[0]: 'SAE_d2_r8',
}

modelResizedBB  = {
    modelNames[0]: True,
}


modelDropoutProb = {
    modelNames[0]: 0.2,
}


## Train one dataset

### Train SEILS

In [ ]:
seilsDatasetLoader = DatasetLoader('SEILS')
train_json, train_img = seilsDatasetLoader.loadTrainPaths()
val_json, val_img = seilsDatasetLoader.loadValPaths()


for modelName in modelNames:
    model = SAE()
    model.set_dropout_probability(modelDropoutProb[modelName])

    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_train = TrainMusicDataset(name='SEILS', jsonPaths=train_json, imagesPaths=train_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=dataset_box_resize)
    dataset_eval = TrainMusicDataset(name='SEILS', jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_train = DataLoader( dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model = trainModel(data_loader_train, data_loader_eval, model, 100, 50, path_checkpoint =  f'/content/drive/MyDrive/TFG/scripts/models/uniq/{modelSavePath[modelName]}_SEILS.pt')


In [ ]:
model_path = '/content/models/SAE_dropout05_redim8.pt'
model_dropout = 0.5
model = torch.load(model_path, map_location=torch.device(device))

model.eval()
model.to(device)

bin_umbrals_to_try = np.arange(0, 1, 0.01)
best_bin_thresholdMean = []
best_bin_thresholdMax = []


best_bin_thresholdMax.append(getBestIoUThreshold(model, type_combination = 'max', model_resize_boxes = True, bin_umbrals_to_try = bin_umbrals_to_try, times_pass_model = times_pass_model, dropout_value=model_dropout))


In [ ]:
matrix_of_iou = np.asarray([[calculate_iou(box_pred, box_true) for box_true in []] for box_pred in []])
matrix_of_iou.size

### Train FMT_M

In [ ]:
fmtmDatasetLoader = DatasetLoader('FMT_M')
train_json, train_img = fmtmDatasetLoader.loadTrainPaths()
val_json, val_img = fmtmDatasetLoader.loadValPaths()


for modelName in modelNames:
    model = SAE()
    model.set_dropout_probability(modelDropoutProb[modelName])

    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_train = TrainMusicDataset(name='FMT_M', jsonPaths=train_json, imagesPaths=train_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=dataset_box_resize)
    dataset_eval = TrainMusicDataset(name='FMT_M', jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_train = DataLoader( dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model = trainModel(data_loader_train, data_loader_eval, model, 100, 50, path_checkpoint =  f'/content/drive/MyDrive/TFG/scripts/models/uniq/{modelSavePath[modelName]}_FMT_M.pt')


### Train FMT_C

In [ ]:

fmtcDatasetLoader = DatasetLoader('FMT_C')
train_json, train_img = fmtcDatasetLoader.loadTrainPaths()
val_json, val_img = fmtcDatasetLoader.loadValPaths()


for modelName in modelNames:
    model = SAE()
    model.set_dropout_probability(modelDropoutProb[modelName])

    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_train = TrainMusicDataset(name='FMT_C', jsonPaths=train_json, imagesPaths=train_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=dataset_box_resize)
    dataset_eval = TrainMusicDataset(name='FMT_C', jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_train = DataLoader( dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model = trainModel(data_loader_train, data_loader_eval, model, 100, 50, path_checkpoint =  f'/content/drive/MyDrive/TFG/scripts/models/uniq/{modelSavePath[modelName]}_FMT_C.pt')

## Eval best umbral

In [ ]:
modelNames = ['bestModel']

modelLoad  = {
    modelNames[0]: 'SAE_d2_r8',
}

modelResizedBB  = {
    modelNames[0]: True,
}

modelPredictDropout  = {
    modelNames[0]: True,
}

modelDropoutProb = {
    modelNames[0]: 0.2,
}

modelCombinationOp  = {
    modelNames[0]: 'mean',
}

modelCombinationNum  = {
    modelNames[0]: 100,
}


### Eval with Capitan

In [ ]:
dataset_name = 'Capitan'
datasetLoader = DatasetLoader(dataset_name)
val_json, val_img = datasetLoader.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{dataset_name}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval = TrainMusicDataset(name=dataset_name, jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/uniq/{modelLoad[modelName]}_{dataset_name}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []


    getBestIoUThreshold(model, model_name=f'{dirToSave}/{dataset_name}-{modelName}-', data_loader_eval=data_loader_eval,
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )

### Eval with SEILS


In [ ]:
dataset_name = 'SEILS'
datasetLoader = DatasetLoader(dataset_name)
val_json, val_img = datasetLoader.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{dataset_name}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval = TrainMusicDataset(name=dataset_name, jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/uniq/{modelLoad[modelName]}_{dataset_name}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []

    getBestIoUThreshold(model, model_name=f'{dirToSave}/{dataset_name}-{modelName}-', data_loader_eval=data_loader_eval,
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )


### Eval with FMT_C

In [ ]:
dataset_name = 'FMT_C'
datasetLoader = DatasetLoader(dataset_name)
val_json, val_img = datasetLoader.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{dataset_name}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval = TrainMusicDataset(name=dataset_name, jsonPaths=val_json, imagesPaths=val_img, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval = DataLoader( dataset_eval, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/uniq/{modelLoad[modelName]}_{dataset_name}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []

# getBestIoUThresholdMultipleDatasets
    getBestIoUThreshold(model, model_name=f'{dirToSave}/{dataset_name}-{modelName}-', data_loader_eval=data_loader_eval,
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )

### Eval with Capitan + SEILS

In [ ]:
datasetName1 = 'Capitan'
datasetLoader1 = DatasetLoader(datasetName1)
val_json1, val_img1 = datasetLoader1.loadValPaths()

datasetName2 = 'SEILS'
datasetLoader2 = DatasetLoader(datasetName2)
val_json2, val_img2 = datasetLoader2.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{datasetName1}+{datasetName2}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval1 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json1, imagesPaths=val_img1, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval1 = DataLoader( dataset_eval1, batch_size=batch_size, shuffle=False, num_workers=0)

     # Load dataset
    dataset_eval2 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json2, imagesPaths=val_img2, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval2 = DataLoader( dataset_eval2, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/comb/{modelLoad[modelName]}_{datasetName1}+{datasetName2}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []

    getBestIoUThresholdMultipleDatasets(model, model_name=f'{dirToSave}/{dataset_name}-{modelName}-', data_loader_eval=[data_loader_eval1, data_loader_eval2],
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )

### Eval with Capitan+FMT_C

In [ ]:
datasetName1 = 'Capitan'
datasetLoader1 = DatasetLoader(datasetName1)
val_json1, val_img1 = datasetLoader1.loadValPaths()

datasetName2 = 'FMT_C'
datasetLoader2 = DatasetLoader(datasetName2)
val_json2, val_img2 = datasetLoader2.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{datasetName1}+{datasetName2}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval1 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json1, imagesPaths=val_img1, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval1 = DataLoader( dataset_eval1, batch_size=batch_size, shuffle=False, num_workers=0)

     # Load dataset
    dataset_eval2 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json2, imagesPaths=val_img2, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval2 = DataLoader( dataset_eval2, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/comb/{modelLoad[modelName]}_{datasetName1}+{datasetName2}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []

    getBestIoUThresholdMultipleDatasets(model, model_name=f'{dirToSave}/{datasetName1}-{datasetName2}-', data_loader_eval=[data_loader_eval1, data_loader_eval2],
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )

### Eval with SEILS+FMT_C

In [ ]:
datasetName1 = 'SEILS'
datasetLoader1 = DatasetLoader(datasetName1)
val_json1, val_img1 = datasetLoader1.loadValPaths()

datasetName2 = 'FMT_C'
datasetLoader2 = DatasetLoader(datasetName2)
val_json2, val_img2 = datasetLoader2.loadValPaths()

dirToSave = f'/content/drive/MyDrive/TFG/valResults{datasetName1}+{datasetName2}'

for modelName in modelNames:
    dataset_box_resize = BBOX_REDIMENSION if modelResizedBB[modelName] else None

    # Load dataset
    dataset_eval1 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json1, imagesPaths=val_img1, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval1 = DataLoader( dataset_eval1, batch_size=batch_size, shuffle=False, num_workers=0)

     # Load dataset
    dataset_eval2 = TrainMusicDataset(name=dataset_name, jsonPaths=val_json2, imagesPaths=val_img2, resize_shape=resized_shape, transforms=get_transform(), box_resize=BBOX_REDIMENSION)

    # Dataset loaders
    data_loader_eval2 = DataLoader( dataset_eval2, batch_size=batch_size, shuffle=False, num_workers=0)

    model_path = f'/content/models/comb/{modelLoad[modelName]}_{datasetName1}+{datasetName2}.pt'
    model = torch.load(model_path, map_location=torch.device(device))

    model.eval()
    model.to(device)

    if modelPredictDropout[modelName]:
        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    bin_umbrals_to_try = np.arange(0, 1, 0.01)
    best_bin_thresholdMean = []
    best_bin_thresholdMax = []

    getBestIoUThresholdMultipleDatasets(model, model_name=f'{dirToSave}/{modelName}-', data_loader_eval=[data_loader_eval1, data_loader_eval2],
                        type_combination=modelCombinationOp[modelName],
                        model_resize_boxes=modelResizedBB[modelName], bin_umbrals_to_try=bin_umbrals_to_try,
                        times_pass_model=modelCombinationNum[modelName], dropout_value=modelDropoutProb[modelName]
                        )

## Test models

In [ ]:
modelNames = ['Capitan', 'SEILS', 'FMT_C', 'Capitan+SEILS', 'Capitan+FMT_C', 'SEILS+FMT_C']



modelLoad  = {
    modelNames[0] : f'uniq/SAE_d2_r8_{modelNames[0]}.pt',
    modelNames[1] : f'uniq/SAE_d2_r8_{modelNames[1]}.pt',
    modelNames[2] : f'uniq/SAE_d2_r8_{modelNames[2]}.pt',
    modelNames[3] : f'comb/SAE_d2_r8_{modelNames[3]}.pt',
    modelNames[4] : f'comb/SAE_d2_r8_{modelNames[4]}.pt',
    modelNames[5] : f'comb/SAE_d2_r8_{modelNames[5]}.pt',
    }


modelResizedBB  = {
    modelName:True for modelName in modelNames
    }

modelPredictDropout  = {
    modelName:True for modelName in modelNames
    }

modelDropoutProb = {
    modelName:0.2 for modelName in modelNames
    }

modelCombinationOp  = {
    modelName:'mean' for modelName in modelNames
    }

modelCombinationNum  = {
    modelName:20 for modelName in modelNames
    }

modelBestUmbral  = {
    modelNames[0]: 0.57,
    modelNames[1]: 0.45,
    modelNames[2]: 0.37,
    modelNames[3]: 0.43,
    modelNames[4]: 0.45,
    modelNames[5]: 0.41,
}

singularModelNames = modelNames[ :3]  # Capitan, SEILS, FMT_C
compostModelNames  = modelNames[3: ]  # All +'s

modelsToTestWithCapitan = ['Capitan']#['SEILS+FMT_C']#['SEILS', 'FMT_C', 'Capitan+SEILS', 'Capitan+FMT_C']
modelsToTestWithSEILS   = ['Capitan+FMT_C']#['Capitan', 'SEILS', 'FMT_C', 'Capitan+SEILS', 'SEILS+FMT_C']
modelsToTestWithFMTC    = ['Capitan+SEILS']#['Capitan', 'SEILS', 'FMT_C', 'Capitan+FMT_C', 'SEILS+FMT_C']

modelsToTestWithCapitan = modelNames
modelsToTestWithSEILS   = modelNames
modelsToTestWithFMTC    = modelNames

In [ ]:

for dataset_name in ['Capitan', 'SEILS', 'FMT_C']:
  datasetLoader = DatasetLoader(dataset_name)

  # datasetLoader.drawBoxesInDataset()

  # Load the datasets
  test_json, test_img = datasetLoader.loadTestPaths()

  # Resized for coco (to check if metrics change)
  dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
  data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0)



  for iteration, batch in enumerate(data_loader_test):
      print()
      print(iteration)
      print()

      # Get the inputs and labels from the batch
      image, target = batch
      # image, targetImage = image.to(device), targetImage.to(device)
      targetBoxes = target.squeeze().numpy().tolist()
      # drawBoxesOnTensor(image, targetBoxes)

      saveBoxesOnTensorToImage(tensor_image=image, bboxes=targetBoxes, image_name=f'/content/drive/MyDrive/TFG/images/{dataset_name}/{iteration}_GT.png')

      torch.cuda.empty_cache()
      gc.collect()
      print()

### Test with Capitan

In [ ]:
dataset_name = 'Capitan'
datasetLoader = DatasetLoader(dataset_name)

# datasetLoader.drawBoxesInDataset()

# Load the datasets
test_json, test_img = datasetLoader.loadTestPaths()

# Resized for coco (to check if metrics change)
dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0)

dirToSave = f'/content/drive/MyDrive/TFG/exp2/testResults{dataset_name}'

for modelName in modelsToTestWithCapitan:
    model_path = f'/content/models/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()
    if modelPredictDropout[modelName]:

        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)

    # Se guarda el fichero en el directorio indicado con el nombre del modelName
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  =  modelCombinationNum[modelName],
              type_combination  =  modelCombinationOp[modelName],
              saveImage = True,
              dataset_name = dataset_name)



### Test with SEILS

In [ ]:
dataset_name = 'SEILS'
datasetLoader = DatasetLoader(dataset_name)

# datasetLoader.drawBoxesInDataset()

# Load the datasets
test_json, test_img = datasetLoader.loadTestPaths()

# Resized for coco (to check if metrics change)
dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0)

dirToSave = f'/content/drive/MyDrive/TFG/exp2/testResults{dataset_name}'

for modelName in modelsToTestWithSEILS:
    model_path = f'/content/models/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()
    if modelPredictDropout[modelName]:

        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)

    # Se guarda el fichero en el directorio indicado con el nombre del modelName
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  =  modelCombinationNum[modelName],
              type_combination  =  modelCombinationOp[modelName],
              saveImage = True,
              dataset_name = dataset_name)



### Test with FMT_C

In [ ]:
dataset_name = 'FMT_C'
datasetLoader = DatasetLoader(dataset_name)

# datasetLoader.drawBoxesInDataset()

# Load the datasets
test_json, test_img = datasetLoader.loadTestPaths()

# Resized for coco (to check if metrics change)
dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0)

dirToSave = f'/content/drive/MyDrive/TFG/exp2/testResults{dataset_name}'

for modelName in modelsToTestWithFMTC:
    model_path = f'/content/models/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()
    if modelPredictDropout[modelName]:

        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)

    # Se guarda el fichero en el directorio indicado con el nombre del modelName
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  =  modelCombinationNum[modelName],
              type_combination  =  modelCombinationOp[modelName],
              saveImage = True,
              dataset_name = dataset_name)



### Test with FMT_M

In [ ]:
dataset_name = 'FMT_M'
datasetLoader = DatasetLoader(dataset_name)

# datasetLoader.drawBoxesInDataset()

# Load the datasets
test_json, test_img = datasetLoader.loadTestPaths()

# Resized for coco (to check if metrics change)
dataset_test = TestMusicDataset(name=dataset_name, jsonPaths=test_json, imagesPaths=test_img, resize_shape=resized_shape, transforms=get_transform())
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0)

dirToSave = f'/content/drive/MyDrive/TFG/exp2/testResults{dataset_name}'

for modelName in modelNames:
    model_path = f'/content/models/{modelLoad[modelName]}'
    model = torch.load(model_path, map_location=torch.device(device))
    print(f'Loading model {modelLoad[modelName]}')
    model.eval()
    if modelPredictDropout[modelName]:

        model.enable_eval_dropout()
        model.set_dropout_probability(modelDropoutProb[modelName])

    model.to(device)

    # Se guarda el fichero en el directorio indicado con el nombre del modelName
    testModel(model,
              model_name = modelName,
              dir_to_save = dirToSave,
              data_loader_test = data_loader_test,
              bin_umbrals_to_try = [0.5, 0.7, modelBestUmbral[modelName]],
              model_resize_boxes = modelResizedBB[modelName],
              times_pass_model  =  modelCombinationNum[modelName],
              type_combination  =  modelCombinationOp[modelName])



# Predict and show eval

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    model.to(device)

    for image, target, targetImage in iter(data_loader_train):
    # saveTensorToImageGrayScale(model(image.to(device)).cpu(), 'dropout3_mean_pass1.jpg')

    # drawTensorImageGrayScale(model(image.to(device)).cpu())

      imagenRes = model(image.to(device)).cpu()
      drawTensorImageGrayScale(imagenRes)
      drawTensorImageGrayScale(binarizarTensor(imagenRes, bin_threshold_percentaje = 0.5))
      # imagen_bin5 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.3)
      # imagen_bin7 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.5)
      # imagen_bin9 = binarizarTensor(imagenRes, bin_threshold_percentaje = 0.7)

      # saveTensorToImageGrayScale(imagenRes, 'bin_prediction.jpg')
      # saveTensorToImageGrayScale(imagen_bin5, 'bin_output3.jpg')
      # saveTensorToImageGrayScale(imagen_bin7, 'bin_output5.jpg')
      # saveTensorToImageGrayScale(imagen_bin9, 'bin_output7.jpg')

    # print('Mean result of 10 passes')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'mean'), 'dropout_10mean_result.jpg')
    # saveTensorToImageGrayScale(forwardToModel(model, image, times_pass_model = 10, type_combination = 'max'), 'dropout_10max_result.jpg')

    # bboxes = getConnectedComponents(model(image.to(device)).cpu())

    # drawBoxesOnTensor(model(image.to(device)).cpu(), bboxes)
    # drawTensorImageGrayScale(targetImage)
    # print(target['boxes'])

In [ ]:
torch.cuda.empty_cache()
gc.collect()

model.to(device)

evalData = iter(data_loader_eval)
next(evalData)
next(evalData)
batch = next(evalData)
image, target, targetImage = batch
imagePass = image.to(device)

model.train()
reconstructed1 = model(imagePass).cpu().detach()
reconstructed2 = model(imagePass).cpu().detach()
print(torch.equal(reconstructed1, reconstructed2))

model.eval()
reconstructed1 = model(imagePass).cpu().detach()
reconstructed2 = model(imagePass).cpu().detach()
print(torch.equal(reconstructed1, reconstructed2))

model.enable_eval_dropout()
reconstructed1 = model(imagePass).cpu().detach()
reconstructed2 = model(imagePass).cpu().detach()
print(torch.equal(reconstructed1, reconstructed2))


In [ ]:
torch.cuda.empty_cache()
gc.collect()

evalData = iter(data_loader_eval)
next(evalData)
next(evalData)
# next(evalData)
# next(evalData)
batch = next(evalData)
image, target, targetImage = batch

drawTensorImageGrayScale(image)
drawTensorImageGrayScale(targetImage)

imagePass = image.to(device)
model.to(device)
reconstructed = model(imagePass)

detached_reconstructed = reconstructed.cpu().detach()

drawTensorImageGrayScale(detached_reconstructed)

targetBoxes = target['boxes'].squeeze().numpy().tolist()

"""No Area check"""
boxes = getConnectedComponents(detached_reconstructed, min_area=0, targetBoxes=targetBoxes)

# drawBoxesPredictedAndGroundTruth(detached_reconstructed, boxes, targetBoxes)

"""Area checking"""
boxes = getConnectedComponents(detached_reconstructed, min_area=100, targetBoxes=targetBoxes)

# drawBoxesPredictedAndGroundTruth(detached_reconstructed, boxes, targetBoxes)
drawBoxesPredictedAndGroundTruth(image, boxes, targetBoxes)


In [ ]:
# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')

# Plotting the last 100 values
losses_plot = [loss_plot.cpu().detach().numpy() for loss_plot in losses[-100:]]
plt.plot(losses_plot)

print(f'ENDED TRAINING')
print()


transformToPil = T.ToPILImage()

print(image.size())
for i, item in enumerate(targetImage):
    item = transformToPil(item.cpu())#.detach().numpy()
    display(item)
    # Reshape the array for plotting
    #item = item.reshape(-1, 512, 512)
    #plt.imshow(item[0])

for i, item in enumerate(reconstructed):
    item = transformToPil(item.cpu())#.detach().numpy()
    display(item)
    #item = item.cpu().detach().numpy()
    #item = item.reshape(-1, 512, 512)
    #plt.imshow(item[0])

In [ ]:


# read the image
#img = cv2.imread('C:\\Users\\samuo\\OneDrive\\Escritorio\\TFG\\scripts\\Capitan\\images\\12608.JPG')
img = cv2.imread('C:\\Users\\samuo\\Downloads\\resultadoAE.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# find the connected components
output = cv2.connectedComponentsWithStats(binary, connectivity=8, ltype=cv2.CV_32S)

# get the labels, stats, and centroids of each connected component
num_labels = output[0]
labels = output[1]
stats = output[2]
centroids = output[3]

for i in range(1, num_labels):
    # draw a rectangle around each connected component
    cv2.rectangle(img, (stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP]),
                  (stats[i, cv2.CC_STAT_LEFT] + stats[i, cv2.CC_STAT_WIDTH],
                   stats[i, cv2.CC_STAT_TOP] + stats[i, cv2.CC_STAT_HEIGHT]),
                  (0, 0, 255), 2)

print(num_labels)
cv2.imwrite('C:\\Users\\samuo\\Downloads\\processed_resultadoAE.jpg', img)

# convert BGR to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image
plt.imshow(img)
plt.show()

In [ ]:

# Load the grayscale image
image = cv2.imread('C:\\Users\\samuo\\Downloads\\resultadoAE.jpg', cv2.IMREAD_GRAYSCALE)
tensor_image = T.ToTensor()(image)
bboxes = getConnectedComponents(tensor_image, bin_threshold_percentaje = 0.5)
drawBoxesOnTensor(tensor_image, bboxes)

In [ ]:

# Get the current date
date_string = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

# Format the date as a number string

# Print the date string
print(date_string)
